In [ ]:

from platform import python_version

print(python_version())

In [1]:
import carla


In [ ]:
import os
os.environ["NGSIM_DIR"] = "C:/Users/Surender Harsha/Downloads/NGSIM/NGSIM"


In [2]:
#import torch
import numpy as np
import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal,Beta,Dirichlet
from torch.distributions import Categorical
from torch.distributions.normal import Normal
import os
#os.environ["NGSIM_DIR"] = "C:\Users\Surender Harsha\Downloads\NGSIM\NGSIM"
#os.environ["NGSIM_DIR"] = ""
#os.environ["OPENDD_DIR"] = "/home/surender/Downloads/openDD"
#os.environ["CARLA_PATH"] = "/home/surender/Downloads/carlaOld"
import sys
#sys.path.append('/home/surender/Downloads/CARLA_0.9.9.4/PythonAPI/carla/dist')
import carla
import random
import argparse

from carla_real_traffic_scenarios.carla_maps import CarlaMaps
from carla_real_traffic_scenarios.ngsim import NGSimDatasets, DatasetMode
from carla_real_traffic_scenarios.ngsim.scenario import NGSimLaneChangeScenario
from carla_real_traffic_scenarios.opendd.scenario import OpenDDScenario
from carla_real_traffic_scenarios.reward import RewardType
from carla_real_traffic_scenarios.scenario import Scenario

from carla_birdeye_view import BirdViewProducer, BirdViewCropType, PixelDimensions
from PIL import Image
#from IPython.display import clear_output, Image, display, HTML
import cv2

%matplotlib tk
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import threading
import time
import math
#import pyro.distributions as dist

In [3]:
from torch.utils.data.sampler import BatchSampler, SubsetRandomSampler

In [4]:
#import tensorflow as tf

In [5]:
#from tensorflow.keras.applications import MobileNet

In [6]:
#mdl = MobileNet(input_shape=(186, 150, 3),include_top=False,weights="imagenet",pooling=max)

In [7]:
#mdl.summary()

In [8]:
if(torch.cuda.is_available()): 
    device = torch.device('cuda:0') 
    torch.cuda.empty_cache()
    print("Device set to : " + str(torch.cuda.get_device_name(device)))
else:
    
    
    print("Device set to : cpu")

Device set to : NVIDIA GeForce GTX 1080 Ti


In [9]:
class RolloutBuffer:
    def __init__(self):
        self.actions = []
        self.states = []
        self.helper = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []
    

    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]
        del self.helper[:]

In [10]:
from controller import VehiclePIDController

In [11]:
#torch.backends.cudnn.deterministic = True

In [12]:
nn.Parameter(torch.ones(1, 2)*-0.5)

Parameter containing:
tensor([[-0.5000, -0.5000]], requires_grad=True)

In [77]:
'''
class Flatten(nn.Module):
    """Helper to flatten a tensor."""
    def forward(self, x):
        return x.view(x.size(0), -1)
'''


class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, has_continuous_action_space, action_std_init,hidden_size = 512,num_channels = 5):
        super(ActorCritic, self).__init__()

        self.has_continuous_action_space = has_continuous_action_space

        if has_continuous_action_space:
            self.action_dim = action_dim
            self.action_var = torch.full((action_dim,), action_std_init * action_std_init).to(device)
        # 186x150
        self.layer1 = nn.Sequential(
            nn.Conv2d(num_channels, 64, kernel_size=12, stride=2), #50x150
            nn.BatchNorm2d(64), # 20x70
            nn.MaxPool2d(4,2), # 9x34
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2),
            nn.BatchNorm2d(128), #4x16
            nn.MaxPool2d(2,1), 
            nn.ReLU(), # 3x15
            nn.Conv2d(128, 256, kernel_size=3, stride=2),
            nn.BatchNorm2d(256), #1x6
            nn.MaxPool2d(1,6),
            nn.ReLU(), #1x1
            
            #nn.ReLU(),
            #nn.Conv2d(16, 32, kernel_size=3, stride=2),
            #nn.ReLU(),
            #nn.AvgPool2d(7,7),
            #nn.Flatten(),
            #nn.AvgPool2d(7),
            nn.Flatten(),
            #nn.Linear(256, 256),
            #nn.Dropout(0.4),
            #nn.ReLU(),
            nn.Linear(512, 256),
            nn.Dropout(0.4),
            nn.ReLU(),
            nn.Linear(256,128)
            #nn.ReLU(),
            #nn.Linear(64,action_dim),
        )
        self.layer2 = nn.Sequential(
        nn.Linear(131, 64),
            nn.Dropout(0.4),
            nn.ReLU(),
            nn.Linear(64,64),
            nn.Dropout(0.4),
            nn.ReLU()
        
        )
        self.actor = nn.Sequential(
            #nn.Linear(131, 64),
            #nn.Dropout(0.4),
            #nn.ReLU(),
            #nn.Linear(64,64),
            #nn.Dropout(0.4),
            #nn.ReLU(),
            nn.Linear(64,action_dim),
            nn.Tanh()
            
            
        )
        self.critic = nn.Sequential(
            #nn.Linear(131, 64),
            #nn.Dropout(0.4),
            #nn.ReLU(),
            #nn.Linear(64,64),
            #nn.Dropout(0.4),
            #nn.ReLU(),
            nn.Linear(64,1),
            nn.Tanh()
            #nn.Conv2d(num_channels, 16, kernel_size=8, stride=4),
            #nn.ReLU(),
            #nn.Conv2d(16, 32, kernel_size=3, stride=2),
            #nn.ReLU(),
            #nn.AvgPool2d(7,7),
            #nn.Flatten(),
            #nn.AvgPool2d(7),
            #nn.Flatten(),
            #nn.Linear(1280, 256),
            #nn.ReLU(),
            #nn.Linear(512, 256),
            #nn.ReLU(),
            #nn.Linear(256,1),
            #nn.ReLU(),
            #nn.Linear(64,action_dim),
            #nn.Tanh()
            #nn.AvgPool2d(7),
            #nn.Flatten(),
            #nn.Linear(1280, 1),
            #nn.ReLU(),
            #nn.Linear(512, 256),
            #nn.ReLU(),
            #nn.Linear(256,64),
            #nn.ReLU(),
            #nn.Linear(64,1),
            #nn.Tanh()
        
        )
        '''
        self.var = nn.Sequential(
            
            nn.Linear(64,64),
            nn.Dropout(0.4),
            nn.ReLU(),
            nn.Linear(64,2),
            nn.Softplus()
        )
        '''
        self.action_log_std = nn.Parameter(torch.ones(1, action_dim) * -0.5)
        '''
        self.actor = nn.Sequential(
            nn.Conv2d(num_channels, 32, kernel_size=8, stride=2),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(4,2),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=1),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2,1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=1),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2,1),
            #nn.Conv2d(128, 256, kernel_size=4, stride=1),
            #nn.BatchNorm2d(256),
            #nn.MaxPool2d(2,1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(124416, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,action_dim),
            nn.Tanh()
            
            
        )
        
        self.critic = nn.Sequential(
            nn.Conv2d(num_channels, 32, kernel_size=8, stride=2),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(4,2),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=1),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2,1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=1),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2,1),
            #nn.Conv2d(128, 256, kernel_size=4, stride=1),
            #nn.BatchNorm2d(256),
            #nn.MaxPool2d(2,1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(124416, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,1),
            nn.Tanh()
            
            
        )
        
        
        #self.l1 = nn.Conv2d(num_channels, 32, kernel_size=4, stride=2)
        #self.l2 = nn.Conv2d(32, 16, kernel_size=4, stride=2)
        #self.l3 = nn.Conv2d(16, 8, kernel_size=3, stride=1)
        #self.p = nn.MaxPool2d(2, 2)
        '''
        ''''
        self.encoder = nn.Sequential(
            nn.Conv2d(num_channels, 32, kernel_size=4, stride=2),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 16, kernel_size=4, stride=2),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 16, kernel_size=3, stride=1),
            
            #nn.MaxPool2d(2, 2)
        )
        
        self.l1 = nn.ConvTranspose2d(16, 16, kernel_size = 8, stride=4)
        self.relu = nn.ReLU()
        self.l2 = nn.ConvTranspose2d(16, 32, kernel_size = 8, stride=1)
        self.l3 = nn.ConvTranspose2d(32, 3, kernel_size = 8, stride=2)
        self.sig = nn.Sigmoid()
        
        self.layer1 = nn.Conv2d(num_channels, 32, kernel_size=8, stride=4)
        self.relu = nn.ReLU();
        self.layer2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.layer3 = nn.Conv2d(64, 32, kernel_size=3, stride=1)
        self.flatten = nn.Flatten()
        self.layer4 = nn.Linear(9120,512)
        self.actol = nn.Linear(512, action_dim)
        self.acto = nn.Tanh()
        self.crit = nn.Linear(512, 1)
        '''
        #self.actor = nn.Sequential(self.main,nn.Linear(hidden_size, action_dim))
        #self.critic = nn.Sequential(self.main_two,nn.Linear(hidden_size, 1))
        '''
        # actor
        if has_continuous_action_space :
            self.actor = nn.Sequential(
                            nn.Linear(state_dim, 64),
                            nn.Tanh(),
                            nn.Linear(64, 64),
                            nn.Tanh(),
                            nn.Linear(64, action_dim),
                            nn.Tanh()
                        )
        else:
            self.actor = nn.Sequential(
                            nn.Linear(state_dim, 64),
                            nn.Tanh(),
                            nn.Linear(64, 64),
                            nn.Tanh(),
                            nn.Linear(64, action_dim),
                            nn.Softmax(dim=-1)
                        )

        
        # critic
        self.critic = nn.Sequential(
                        nn.Linear(state_dim, 64),
                        nn.Tanh(),
                        nn.Linear(64, 64),
                        nn.Tanh(),
                        nn.Linear(64, 1)
                    )
        '''
        
    def set_action_std(self, new_action_std):

        if self.has_continuous_action_space:
            #del self.action_var
            self.action_var = torch.full((self.action_dim,),new_action_std * new_action_std).to(device)
        else:
            print("--------------------------------------------------------------------------------------------")
            print("WARNING : Calling ActorCritic::set_action_std() on discrete action space policy")
            print("--------------------------------------------------------------------------------------------")

    def AutoEncoder(self,x):
        '''
        x = self.encoder(x)
        print(x.shape)
        x = self.l1(x)
        x = self.relu(x)
        print(x.shape)
        x = self.l2(x)
        x = self.relu(x)
        print(x.shape)
        x = self.l3(x)
        x = self.sig(x)
        print(x.shape)
        
        print(x.shape)
        x = self.l1(x)
        print(x.shape)
        x = self.p(x)
        print(x.shape)
        x = self.l2(x)
        print(x.shape)
        x = self.p(x)
        print(x.shape)
        x = self.l3(x)
        print(x.shape)
        x = self.p(x)
        print(x.shape)
        '''
        #x = self.encoder(x)
        #return x
        pass
        

    def forward(self):
        raise NotImplementedError
        '''
        print(x.shape)
        x = self.layer1(x)
        #print(x.shape)
        x = self.relu(x)
        print(x.shape)
        x = self.layer2(x)
        #print(x.shape)
        x = self.relu(x)
        print(x.shape)
        x = self.layer3(x)
        #print(x.shape)
        x = self.relu(x)
        print(x.shape)
        x = self.flatten(x)
        print(x.shape)
        x = self.layer4(x)
        print(x.shape)
        a = self.actol(x)
        a = self.acto(a)
        v = self.crit(x)
        return a,v
        '''
        
    def backward(self, x):
        import pdb
        pdb.set_trace()
        return x
    

    def act(self, state,helper,val = False):
        
        if self.has_continuous_action_space:
            #x = self.main(state)
            inps1 = state
            inps2 = helper
            action_m = self.layer1(inps1)
            #print(action_m,inps2,action_m.shape,inps2.shape)
            inps = torch.cat((action_m,inps2),1)
            x = self.layer2(inps)
            action_mean = self.actor(x)
            if val:
                return action_mean.detach(),-1
            action_log_std = self.action_log_std.expand_as(action_mean)
            
            #action_var = self.var(x) + 0.001
            #action_std = self.std(state)
            #print(action_std)
            #self.set_action_std(torch.mean(action_std))
            #print(action_mean,action_std)
            #action_var = action_var.expand_as(action_mean)
            #cov_mat = torch.diag_embed(action_var)
            #dist = MultivariateNormal(action_mean,cov_mat)
            #cov_mat = torch.diag(self.action_var).unsqueeze(dim=0)
            #print(action_mean.view(1, ).data, action_var.view(1, ).data)
            #dist = Beta(action_mean, action_var)
            dist = Normal(action_mean,action_log_std.exp())
        else:
            inps1 = state[:,:-3]
            inps2 = state[:,-3:]
            action_m = self.layer1(inps1)
            #print(action_m,inps2,action_m.shape,inps2.shape)
            inps = torch.cat((action_m,inps2),1)
            
            action_probs = self.actor(inps)
            dist = Categorical(action_probs)

        action = dist.sample()
        
        action_logprob = dist.log_prob(action)
        #action = action*2 - 1
        
        return action.detach(), action_logprob.detach()
    

    def evaluate(self, state,helper, action):
        #print("NOTHING")
        if self.has_continuous_action_space:
            '''
            if len(state.shape) == 3:
                state = state.reshape((1,5,186,150))
            '''
            #x = self.main(state)
            #print(state.shape)
            inps1 = state
            inps2 = helper
            action_m = self.layer1(inps1)
            inps = torch.cat((action_m,inps2),1)
            x = self.layer2(inps)
            
            action_mean = self.actor(x)
            action_log_std = self.action_log_std.expand_as(action_mean)
            #action_var = self.var(x) + 1
            #action_std = self.std(state)
            #print(action_std)
            #print(torch.mean(action_std))
            #self.set_action_std(torch.mean(action_std))
            #var = torch.mean(action_std)
            #print(action_mean.shape)
            #print(action_mean.shape,action_var.shape)
            #action_var = action_var.expand_as(action_mean)
            #print(action_var,action_var.shape)
            #print(action_mean.shape,action_var.shape)
            #cov_mat = torch.diag_embed(action_var)
            #print(cov_mat)
            dist = Normal(action_mean,action_log_std.exp())
            #dist = Beta(action_mean, action_var)
            #print(action_mean.shape,cov_var.shape)
            #dist = MultivariateNormal(action_mean,cov_mat)
            #print(dist)
            # for single action continuous environments
            #if self.action_dim == 1:
            #    action = action.reshape(-1, self.action_dim)

        else:
            inps1 = state[:,:-3]
            inps2 = state[:,-3:]
            action_m = self.layer1(inps1)
            inps = torch.cat((action_m,inps2),1)
            #x = self.layer2(inps)
            action_probs = self.actor(inps)
            dist = Categorical(action_probs)
        #action = (action + 1)/2
        action_logprobs = dist.log_prob(action)
        #print(action_logprobs.shape)
        dist_entropy = dist.entropy()
        inps1 = state
        inps2 = helper
        critic_m = self.layer1(inps1)
        inps = torch.cat((critic_m,inps2),1)
        #action_mean = self.actor(inps)
        x = self.layer2(inps)
        state_values = self.critic(x)
        #print(state_values,action_mean)
        return action_logprobs, state_values, dist_entropy


In [14]:
for i in p.policy.action_log_std:
    print(i)

NameError: name 'p' is not defined

In [50]:
class PPO:
    def __init__(self, state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, has_continuous_action_space, action_std_init=0.6):

        self.has_continuous_action_space = has_continuous_action_space

        if has_continuous_action_space:
            self.action_std = action_std_init
        self.entropy_c = 0.01
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.buffer = RolloutBuffer()

        self.policy = ActorCritic(state_dim, action_dim, has_continuous_action_space, action_std_init).to(device)
        self.optimizer = torch.optim.Adam([
                        {'params': self.policy.layer1.parameters(), 'lr': lr_actor},
                        {'params': self.policy.layer2.parameters(), 'lr': lr_actor},
                        {'params': self.policy.actor.parameters(), 'lr': lr_actor},
                        {'params': self.policy.critic.parameters(), 'lr': lr_critic},
                        {'params': self.policy.action_log_std, 'lr': 0.01}
                        
                        
                    ])
        

        self.policy_old = ActorCritic(state_dim, action_dim, has_continuous_action_space, action_std_init).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.MseLoss = nn.MSELoss()


    def set_action_std(self, new_action_std):
        
        if self.has_continuous_action_space:
            self.action_std = new_action_std
            self.policy.set_action_std(new_action_std)
            self.policy_old.set_action_std(new_action_std)
        
        else:
            print("--------------------------------------------------------------------------------------------")
            print("WARNING : Calling PPO::set_action_std() on discrete action space policy")
            print("--------------------------------------------------------------------------------------------")


    def decay_action_std(self, action_std_decay_rate, min_action_std):
        #print("--------------------------------------------------------------------------------------------")

        if self.has_continuous_action_space:
            self.action_std = self.action_std - action_std_decay_rate
            self.action_std = round(self.action_std, 4)
            if (self.action_std <= min_action_std):
                self.action_std = min_action_std
                #print("setting actor output action_std to min_action_std : ", self.action_std)
            else:
                pass
                #print("setting actor output action_std to : ", self.action_std)
            self.set_action_std(self.action_std)

        else:
            print("WARNING : Calling PPO::decay_action_std() on discrete action space policy")

        #print("--------------------------------------------------------------------------------------------")


    def select_action(self, state,val = False):

        if self.has_continuous_action_space:
            with torch.no_grad():
                helper = torch.FloatTensor(state[1]).to(device)
                state = torch.FloatTensor(state[0]).to(device)
                
                action, action_logprob = self.policy_old.act(state,helper,val)
            if not val:
                self.buffer.states.append(state)
                self.buffer.helper.append(helper)
                self.buffer.actions.append(action)
                self.buffer.logprobs.append(action_logprob)

            return action.detach().cpu().numpy().flatten()

        else:
            with torch.no_grad():
                state = torch.FloatTensor(state).to(device)
                action, action_logprob = self.policy_old.act(state)
            
            self.buffer.states.append(state)
            self.buffer.actions.append(action)
            self.buffer.logprobs.append(action_logprob)

            return action.item()


    def update(self):

        # Monte Carlo estimate of returns
        rewards = []
        discounted_reward = 0
        #print(self.buffer.rewards)
        for reward, is_terminal in zip(reversed(self.buffer.rewards), reversed(self.buffer.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
        #print("I",rewards)
        # Normalizing the rewards
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        #print(rewards,rewards.mean(),rewards.std(unbiased = False))
        rewards = (rewards - rewards.mean()) / (rewards.std(unbiased = False) + 1e-7)
        #print(rewards)
        # convert list to tensor
        old_states = torch.squeeze(torch.stack(self.buffer.states, dim=0)).detach().to(device)
        old_actions = torch.squeeze(torch.stack(self.buffer.actions, dim=0)).detach().to(device)
        old_logprobs = torch.squeeze(torch.stack(self.buffer.logprobs, dim=0)).detach().to(device)
        old_helper = torch.squeeze(torch.stack(self.buffer.helper, dim=0)).detach().to(device)
        batch_size = 32
        print(old_states.shape,old_actions.shape,old_logprobs.shape)
        # Optimize policy for K epochs
        for _ in range(self.K_epochs):
            for index in BatchSampler(SubsetRandomSampler(range(len(self.buffer.states))), batch_size, False):
            # Evaluating old actions and values
                #print(index,old_actions[index],old_actions)
                
                logprobs, state_values, dist_entropy = self.policy.evaluate(old_states[index],old_helper[index], old_actions[index])
                temp_size = logprobs.shape[0]
                #print(state_values.shape)
                #print(logprobs.shape,logprobs)
                # match state_values tensor dimensions with rewards tensor
                state_values = torch.squeeze(state_values)
                #print(state_values.shape)
                #state_values = torch.
                # Finding the ratio (pi_theta / pi_theta__old)
                ratios = torch.exp(logprobs - old_logprobs[index].detach())
                #print(ratios)
                # Finding Spurrogate Loss
                #print(rewards[index].shape)
                advantages = rewards[index] - state_values.detach()   
                #print(rewards)
                #print(ratios.shape,advantages.shape)
                advantages = advantages.reshape(temp_size,1)
                surr1 = ratios * advantages
                surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages
                #print(state_values)
                #print(state_values.shape)
                # final loss of clipped objective PPO
                #print(rewards.shape,state_values.shape)
                loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards[index]) - 0.01*dist_entropy
                #print(surr1,surr2,state_values,rewards,dist_entropy,loss)
                # take gradient step
                self.optimizer.zero_grad()
                loss.mean().backward()
                self.optimizer.step()
            #self.optimizer2.step()
        if self.entropy_c > 0.001:
            self.entropy_c -= 0.0001
        # Copy new weights into old policy
        self.policy_old.load_state_dict(self.policy.state_dict())

        # clear buffer
        self.buffer.clear()
    
    
    def save(self, checkpoint_path):
        torch.save(self.policy_old.state_dict(), checkpoint_path)
   

    def load(self, checkpoint_path):
        self.policy_old.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))
        self.policy.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))

In [18]:
torch.seed()

1214589967405923001

In [52]:
update_timestep = 1000     # update policy every n timesteps
K_epochs = 100             # update policy for K epochs
eps_clip = 0.1             # clip parameter for PPO
gamma = 0.99                # discount factor

lr_actor = 0.001       # learning rate for actor network
lr_critic = 0.001       # learning rate for critic network

random_seed = 0 

In [53]:
def prepare_ngsim_scenario(client: carla.Client, data_mode = "train") -> Scenario:
    data_dir = os.environ.get("NGSIM_DIR")
    #data_dir = os.listdir('/home/surender/Downloads/NGSIM')
    assert data_dir, "Path to the directory with NGSIM dataset is required"
    ngsim_map = NGSimDatasets.list()
    ngsim_dataset = ngsim_map[1]
    client.load_world(ngsim_dataset.carla_map.level_path)
    if data_mode == "train":
        return NGSimLaneChangeScenario(
            ngsim_dataset,
            dataset_mode=DatasetMode.TRAIN,
            data_dir=data_dir,
            reward_type=RewardType.DENSE,
            client=client,
        )
    else:
        return NGSimLaneChangeScenario(
            ngsim_dataset,
            dataset_mode=DatasetMode.VALIDATION,
            data_dir=data_dir,
            reward_type=RewardType.DENSE,
            client=client,
        )

'''
def prepare_opendd_scenario(client: carla.Client) -> Scenario:
    data_dir = os.environ.get("OPENDD_DIR")
    assert data_dir, "Path to the directory with openDD dataset is required"
    maps = ["rdb1", "rdb2", "rdb3", "rdb4", "rdb5", "rdb6", "rdb7"]
    map_name = random.choice(maps)
    carla_map = getattr(CarlaMaps, map_name.upper())
    client.load_world(carla_map.level_path)
    return OpenDDScenario(
        client,
        dataset_dir=data_dir,
        dataset_mode=DatasetMode.TRAIN,
        reward_type=RewardType.DENSE,
        place_name=map_name,
    )

'''
def prepare_ego_vehicle(world: carla.World) -> carla.Actor:
    car_blueprint = world.get_blueprint_library().find("vehicle.audi.a2")

    # This will allow external scripts like manual_control.py or no_rendering_mode.py
    # from the official CARLA examples to take control over the ego agent
    car_blueprint.set_attribute("role_name", "hero")

    # spawn points doesnt matter - scenario sets up position in reset
    ego_vehicle = world.spawn_actor(
        car_blueprint, carla.Transform(carla.Location(0, 0, 500), carla.Rotation())
    )

    assert ego_vehicle is not None, "Ego vehicle could not be spawned"

    # Setup any car sensors you like, collect observations and then use them as input to your model
    return ego_vehicle

In [21]:
def cmd_carla():
    os.system("singularity exec --nv /data/s4120310/Singularity.sif /bin/bash /home/carla/CarlaUE4.sh -opengl")

In [ ]:
def cmd_carla():
    os.system("DISPLAY= /home/surender/Downloads/carlaOld/CarlaUE4.sh -benchmark -fps=10 -quality-level=Low -opengl -Resx=4 -Resy=4 -NoVSync")

In [22]:
thp = threading.Thread(target = cmd_carla)
thp.start()


In [ ]:
thp.join()

In [26]:
host = "localhost"
port = 2000
client = carla.Client(host,port)

In [ ]:
#torch.squeeze(torch.tensor([[1.4]]))

In [ ]:
Beta(1,3).sample()

In [27]:
scenario = prepare_ngsim_scenario(client)

Switch synchronous_mode=True
Change fixed_delta_seconds=0.1


In [31]:
world = client.get_world()
spectator = world.get_spectator()
ego_vehicle = prepare_ego_vehicle(world)


In [ ]:
settings = world.get_settings()
settings.no_rendering_mode = True
world.apply_settings(settings)

In [ ]:
input_data= []

In [ ]:
current_frame = 0

In [ ]:
c = torch.arange(64).reshape(32,2)

In [ ]:
torch.diag(c).unsqueeze(dim=0)

In [ ]:
c.shape

In [ ]:
c

In [ ]:
c[1:,:]

In [ ]:
b = torch.arange(64).reshape(32,2)

In [ ]:
b.shape

In [ ]:
import pyro.distributions.Beta

In [ ]:
b[1:]

In [ ]:
d =Beta(b[1:].float(),c[1:].float())

In [ ]:
d.sample()

In [ ]:
Dirichlet(b)

In [ ]:
d = c.expand_as(b)

In [ ]:
a = torch.diag_embed(d)

In [ ]:
a.float().shape

In [ ]:
a[0][1][1] =1

In [ ]:
a.shape

In [ ]:
a.unsqueeze(dim=0).shape

In [ ]:
b.shape

In [ ]:
a.shape

In [ ]:
dist = MultivariateNormal(b.float(),a.float())

In [ ]:
dist.sample()

In [ ]:
b

In [ ]:
current_frame = 0
input_data = 0

In [ ]:
def check_img(img):
    global input_data,current_frame
    c_img = img
    #print(img.frame)
    array = np.frombuffer(img.raw_data, dtype=np.dtype("uint8"))
    #print(array.shape)
    array = np.reshape(array, (img.height, img.width, 4)) # RGBA format
    array = array[:, :, :3] #  Take only RGB
    #print(array.shape)
    #plt.imshow(array)
    
    img = Image.fromarray(array)
    
    #print(img)
    img = img.resize((320,320), Image.ANTIALIAS)
    #print(img)
    input_data = np.array(img)
    current_frame = c_img.frame

In [ ]:
cam_bp = world.get_blueprint_library().find('sensor.camera.rgb')
cam_bp.set_attribute("image_size_x",str(320))
cam_bp.set_attribute("image_size_y",str(320))
cam_bp.set_attribute("fov",str(100))
cam_location = carla.Location(2,0,1)
cam_rotation = carla.Rotation(0,0,0)
cam_transform = carla.Transform(cam_location,cam_rotation)
ego_front_cam = world.spawn_actor(cam_bp,cam_transform,attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
#self.rgb_front_listener = ego_cam
ego_front_cam.listen(lambda image: check_img(image))

In [ ]:
from carla_real_traffic_scenarios.artificial_lane_change.scenario import ArtificialLaneChangeScenario

In [ ]:
#als = ArtificialLaneChangeScenario(client = client,cmd_for_changing_lane=r.chauffeur_cmd,speed_range_token="HIGHWAY")

In [ ]:
for i in list(world.get_actors()):
    i.destroy()

In [ ]:
world.tick()

In [ ]:
plt.imshow(input_data)

In [ ]:
list(world.get_actors())

In [ ]:
len(list(world.get_actors()))

In [ ]:
action_space = [[0., 0.], [-0.3,0.], [-0.15,0.], [0.25,0.], [0.5, 0.], [0.7, 0.],
                [0., 0.25], [-0.3,0.25], [-0.15,0.25], [0.25,0.25], [0.5, 0.25], [0.7, 0.25],
                [0., -0.25], [-0.3,-0.25], [-0.15,-0.25], [0.25,-0.25], [0.5, -0.25], [0.7, -0.25],
                [0., 0.5], [-0.3,0.5], [-0.15,0.5], [0.25,0.5], [0.5, 0.5], [0.7, 0.5],
                [0., -0.5], [-0.3,-0.5], [-0.15,-0.5], [0.25,-0.5], [0.5, -0.5], [0.7, -0.5]]

In [ ]:
len(action_space)

In [32]:

scenario.reset(ego_vehicle)

In [ ]:
r = scenario.step(ego_vehicle)

In [ ]:
r

In [ ]:
Normal()

In [33]:
c = world.tick()

In [ ]:
v = scenario._veh.speed

In [ ]:
v

In [ ]:
for veh in v:
    if veh.id == scenario._lane_change.vehicle_id:
        c = veh
        

In [ ]:
scenario._target_lane_waypoint.transform.location.x

In [ ]:
scenario._target_lane_waypoint.transform.location.y

In [ ]:
c

In [ ]:
pid = VehiclePIDController(ego_vehicle)

In [ ]:
k = pid.run_step(30,r.info["scenario_data"]["original_veh_transform"])

In [ ]:
k.steer

In [ ]:
ego_vehicle.apply_control(carla.VehicleControl(throttle=0.4, steer=0.5))

In [ ]:
world.tick()

In [ ]:
scenario.step(ego_vehicle)

In [ ]:
world.tick()

In [ ]:
ego_vehicle.get_control().steer

In [ ]:
r.chauffeur_cmd

In [ ]:
r.reward

In [ ]:
e_v = r.info["scenario_data"]["ego_veh"]

In [ ]:
r.done

In [ ]:
e_v.get_location().y

In [ ]:
r.info["scenario_data"]["original_veh_transform"].location

In [ ]:
scenario.close()

In [ ]:
scenario = prepare_ngsim_scenario(client)

In [ ]:
world = client.get_world()
#spectator = world.get_spectator()
ego_vehicle = prepare_ego_vehicle(world)

In [ ]:
scenario.reset(ego_vehicle)

In [ ]:
r

In [ ]:
scenario._target_lane_waypoint.transform.location.x

In [ ]:
ego_vehicle.get_location().y

In [ ]:
way.location.x -= 100

In [ ]:
way.location.x

In [ ]:
ego_vehicle.get_location().x

In [ ]:
way.location.y += 0.5

In [ ]:
ego_vehicle.get_transform

In [ ]:
way.location.y

In [ ]:
def way_cal(ego_vehicle,val,stabilize = False, actual_waypoint = None):
    way = ego_vehicle.get_transform()
    if (val == 0  or val ==1):
        way.location.x += 100        
    if (val == 2 or val == 5):
        way.location.x += 30
        way.location.y += 7
    if (val == 3 or val == 4):
        way.location.x += 30
        way.location.y -= 7
    return way

In [ ]:
r.info

In [ ]:
import random

In [ ]:
list(world.get_actors())[0].id

In [ ]:
for i in range(20):
    #scenario.reset(ego_vehicle)
    #client.apply_batch([carla.command.DestroyActor(x) for x in list(world.get_actors())])
    #world.tick()
    ids = list(world.get_actors())[0].id
    carla.command.DestroyActor(ids)
    world = client.get_world()
    #spectator = world.get_spectator()
    ego_vehicle = prepare_ego_vehicle(world)
    scenario.reset(ego_vehicle)
    done = False
    c = world.tick()
    way = ego_vehicle.get_transform()
    cmd_buffer = [0]
    stab = False
    aw = None
    speed = 40
    while not done:
        pid = VehiclePIDController(ego_vehicle)
        speed = random.randint(30,50)
        k = pid.run_step(speed,way)
        ego_vehicle.apply_control(carla.VehicleControl(throttle=k.throttle, steer=k.steer, brake = k.brake))
        r = scenario.step(ego_vehicle)
        print(r.reward,done,r.chauffeur_cmd)
        #way = scenario._target_lane_waypoint.transform
        way = r.info["scenario_data"]["original_veh_transform"]
        cmd_buffer.append(r.chauffeur_cmd.value)

        if len(cmd_buffer)>5:
            if sum(cmd_buffer[-5:]) == 0 or sum(cmd_buffer[-5:]) == 5:
                way = scenario._target_lane_waypoint.transform
                way.location.x += 50
                speed = 40

        birdview = birdview_producer.produce(

                agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
                )
        rgb = BirdViewProducer.as_rgb(birdview)
        cv2.imshow('Frame',rgb)
        done = r.done
        if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
        c = world.tick()
    cv2.destroyAllWindows()
    #time.sleep(3)

In [ ]:
r

In [ ]:
birdview = birdview_producer.produce(
            agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
            )
rgb = BirdViewProducer.as_rgb(birdview)
cv2.imshow('Frame',rgb)

if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()

In [ ]:
scenario._target_lane_waypoint.transform.location.y

In [ ]:
c

In [ ]:
world.tick()

In [ ]:
plt.imshow(input_data)

In [ ]:
current_frame

In [78]:
p = PPO(1,2,lr_actor,lr_critic,gamma,K_epochs,eps_clip,True)

In [ ]:
del p

In [ ]:
p.load("Model_CHK4.mdl")

In [79]:
print(p.policy)

ActorCritic(
  (layer1): Sequential(
    (0): Conv2d(5, 64, kernel_size=(12, 12), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): MaxPool2d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU()
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
    (7): ReLU()
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): MaxPool2d(kernel_size=1, stride=6, padding=0, dilation=1, ceil_mode=False)
    (11): ReLU()
    (12): Flatten()
    (13): Linear(in_features=512, out_features=256, bias=True)
    (14): Dropout(p=0.4, inplace=False)
    (15): ReLU()
    (16): Linear(in_features=256, out_features=128, bias=True)
  )
  (l

In [ ]:
for i in p.policy.parameters():
    print(i)

In [ ]:
torch.nn.init.orthogonal_(p.policy.layer1.weights)

In [ ]:
def init_normal(m):
    if type(m) == nn.Linear:
        #print("DONE",m.weight)
        nn.init.orthogonal_(m.weight)
        #print(m.weight)

In [ ]:
p.policy.layer1.apply(init_normal)

In [ ]:
p.policy.layer2.apply(init_normal)

In [ ]:
p.policy.actor.apply(init_normal)

In [ ]:
p.policy.critic.apply(init_normal)

In [ ]:
#p.policy.layer1.apply(init_normal)
#p.policy.layer2.apply(init_normal)#
#p.policy.actor.apply(init_normal)
#p.policy.critic.apply(init_normal)

In [ ]:
p.policy.action_log_std

In [ ]:
ae = AutoEncoder(3).to(device)

In [ ]:
img = ae(state)

In [ ]:
img = img.detach().cpu().numpy()

In [ ]:
img = img.reshape((3,320,320))

In [ ]:
img = img.reshape((320,320,3))

In [ ]:
plt.imshow(img)

In [ ]:
from torchsummary import summary

In [46]:
birdview_producer = BirdViewProducer(
    client,  # carla.Client
    target_size=PixelDimensions(width=50, height=150),
    pixels_per_meter=4
    #crop_type=BirdViewCropType.FRONT_AREA_ONLY
    )

In [47]:
birdview = birdview_producer.produce(
            agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
            )

In [48]:
rgb = BirdViewProducer.as_rgb(birdview)
cv2.imshow('Frame',rgb)

if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()

In [56]:
cv2.destroyAllWindows()

In [ ]:
plt.imshow(a[:,:,2])

In [ ]:
a = rgb/255.

In [ ]:
mn=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

In [ ]:
for i in range(3):
    a[:,:,i] = (a[:,:,i] - mn[i])/std[i]

In [ ]:
plt.imshow(a)

In [ ]:
a =rgb/255.

In [70]:
a = a.reshape(1,5,50,150)

In [72]:
a.shape

(1, 5, 50, 150)

In [76]:
p.policy.layer1(in_data).shape

torch.Size([1, 512])

In [ ]:
a = a.reshape(1,3,224,224)

In [ ]:
with torch.no_grad():
    ins = torch

In [74]:
with torch.no_grad():
    in_data = torch.FloatTensor(a).to(device)

In [ ]:
cs = mobilenet(in_data)

In [ ]:
torch.argmax(cs)

In [ ]:
cv2.destroyAllWindows()


In [ ]:
plt.imshow(birdview[1])

In [ ]:
plt.imshow(birdview[3])

In [ ]:
plt.imshow(birdview[4])

In [ ]:
birdview[1].shape

In [ ]:
a = birdview[0].reshape(1,186,150)
a = np.append(a,birdview[1].reshape(1,186,150),axis=0)
a = np.append(a,birdview[2].reshape(1,186,150),axis=0)
a = np.append(a,birdview[3].reshape(1,186,150),axis=0)
a = np.append(a,birdview[4].reshape(1,186,150),axis=0)

In [ ]:
a.shape

In [ ]:
birdview[0]

In [ ]:
plt.imshow(birdview[4])

In [ ]:
#a[1,:,:] = birdview[3]

In [ ]:
a = rgb.reshape(1,3,186,150)

In [ ]:
a = a/255.

In [ ]:
plt.imshow(a.reshape(3,186,150))

In [ ]:
a = np.append(a,birdview[3].reshape(1,224,224),axis=0)
a = np.append(a,birdview[4].reshape(1,224,224),axis=0)

In [ ]:
plt.imshow(a[2])

In [ ]:
a = a.reshape(1,5,186,150)

In [ ]:
a.shape

In [ ]:
inps.shape

In [ ]:
device

In [ ]:
with torch.no_grad():
    inps = torch.FloatTensor(a).to(device)
    c = p.policy.layer1(inps)
    #b = nn.Flatten()(c)
    #d = nn.ReLU(b)

In [ ]:
c.shape

In [ ]:
a.shape

In [ ]:
a = rgb.reshape(1,3,224, 224)

In [ ]:
cd =  a

In [ ]:
cd.shape

In [ ]:
a = cd

In [ ]:
ap = np.concatenate((in_data[0],inps)).reshape(1,1283)

In [ ]:
inps = [0,1,0]

In [ ]:
np.hstack((a,inps))

In [ ]:
np.array(inps).reshape(186,150)

In [ ]:
a[0][0][0]

In [ ]:
with torch.no_grad():
    xd = torch.FloatTensor(inputs).to(device)

In [ ]:
a = np.concatenate((a[0,:,:],inps))

In [ ]:
inputs = [a,np.array(inps).reshape(1,3)]

In [ ]:
a.shape

In [ ]:
inputs[1]

In [ ]:
p.select_action(inputs)

In [ ]:
p.select_action(inputs,True)

In [ ]:
p.policy.action_log_std

In [ ]:
np.append(a,inps,axis=0)

In [ ]:
inp = rgb.reshape(1,186,150,3)

In [ ]:
inp = np.append(inp, rgb.reshape(1,186,150,3),axis = 0)

In [ ]:
inp.shape

In [ ]:
cv2.destroyAllWindows()

In [ ]:
action_var = torch.full((2,), 0.9 * 0.9).to(device)

In [ ]:
action_var

In [ ]:
state = torch.FloatTensor(a).to(device)

In [ ]:
state.shape

In [ ]:
action_mean = p.policy.actor(state)


In [ ]:
action_mean

In [ ]:
ego_vehicle.get_transform().rotation.yaw

In [ ]:
import math
v_begin = ego_vehicle.get_transform().location
v_end = v_begin + carla.Location(x=math.cos(math.radians(ego_vehicle.get_transform().rotation.yaw)),
                                         y=math.sin(math.radians(ego_vehicle.get_transform().rotation.yaw)))

In [ ]:
v_end.x,v_end.y

In [ ]:
v_begin.x,v_begin.y

In [ ]:
v_vec = np.array([v_end.x - v_begin.x, v_end.y - v_begin.y, 0.0])

In [ ]:
v_vec

In [ ]:
cov_mat = torch.diag(action_var).unsqueeze(dim=0)


In [ ]:
cov_mat

In [ ]:
dist = MultivariateNormal(action_mean, cov_mat)

In [ ]:
dist.entropy()

In [ ]:
action = dist.sample()


In [ ]:
action

In [ ]:
action

In [ ]:
action_logprob = dist.log_prob(action)

In [ ]:
action_logprob

In [ ]:
val = 0
mn=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

rgb = BirdViewProducer.as_rgb(birdview)/255.
#in_data = a.reshape(1,5,186,150)
for i in range(3):
    rgb[:,:,i] = (rgb[:,:,i] - mn[i])/std[i]
in_data = rgb.reshape(1,3,224,224)

with torch.no_grad():
    in_data = torch.FloatTensor(in_data).to(device)
    in_data = mobilenet.features(in_data)
    in_data = nn.AvgPool2d(7,7)(in_data)
    in_data = nn.Flatten()(in_data)
    in_data = in_data.detach().cpu().numpy()
#in_data = mdl(in_data).numpy()
#in_data = in_data.reshape(in_data.shape[0],62720)
#in_data = in_data.reshape((1,5,186,150))
#in_data = input_data.reshape((1,3,320,320))
#print(in_data.shape)
if val == 0 or val == 1:
    inps= [0,0,1]
if val == 2 or val == 5:
    inps = [0,1,0]
if val == 4 or val ==3:
    inps = [1,0,0]
inps = np.array(inps)
ap = np.concatenate((in_data[0],inps)).reshape(1,1283)

In [ ]:
mobilenet.classifier

In [ ]:
with torch.no_grad():
    in_data = torch.FloatTensor(in_data).to(device)
    c = mobilenet.classifier(in_data)

In [ ]:
np.argmax(c.detach().cpu().numpy())

In [ ]:
mobilenet.classifier(in_data)

In [ ]:
action_space[p.select_action(ap)]

In [ ]:
b = mdl(inp).numpy()

In [ ]:
b.shape

In [ ]:
a.shape

In [ ]:
b.shape

In [ ]:
a.reshape(a.shape[0],20480).shape

In [ ]:
b.reshape(b.shape[0],20480).shape

In [ ]:
rgb.shape

In [ ]:
ego_vehicle.get_location().y 

In [ ]:
scenario._target_lane_waypoint.transform.location.y - ego_vehicle.get_location().y 

In [ ]:
scenario._target_lane_waypoint.transform.location.x - ego_vehicle.get_location().x 

In [ ]:
ego_vehi

In [ ]:
v = ego_vehicle.get_velocity()

In [ ]:
v.z

In [ ]:
v.x

In [ ]:
v.y

In [ ]:
ego_vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer=0.0))


In [ ]:
scenario.step(ego_vehicle)

In [ ]:
world.tick()

In [ ]:
birdview = birdview_producer.produce(
            agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
            )
rgb = BirdViewProducer.as_rgb(birdview)
cv2.imshow('Frame',rgb)

if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()


In [ ]:
cv2.destroyAllWindows()

In [ ]:
plt.imshow(birdview[0]) #Full Road Greyed out

In [ ]:
plt.imshow(birdview[1])  #Lanes

In [ ]:
plt.imshow(birdview[2]) #Centerlines

In [ ]:
plt.imshow(birdview[3])#Other vehicles

In [ ]:
plt.imshow(birdview[4])# Ego agent

In [ ]:
in_data = birdview[:5,:,:]

In [ ]:
in_data.shape

In [ ]:
in_data = in_data.reshape((1,5,186,150))

In [ ]:
np.max(in_data)

In [ ]:
in_data.shape

In [ ]:
p.select_action(state)

In [ ]:
p.policy.actor(state.to(device)).shape

In [ ]:
p.policy.forward(torch.FloatTensor(in_data).to(device))

In [ ]:
import math

In [ ]:
p.policy.actor.parameters()

In [ ]:
with torch.no_grad():
    state = torch.FloatTensor(in_data)

In [ ]:
state

In [ ]:
state.shape

In [ ]:
k = p.policy.AutoEncoder(state)

In [ ]:
ae = AutoEncoder(3).to(device)
optimizer = torch.optim.Adam([
                        {'params': ae.parameters(), 'lr': 0.003}
])

criterion = nn.MSELoss()



In [ ]:
inp = torch.cat((state,state), 0)

In [ ]:
inp.shape[0]

In [ ]:
del inp_tensor,outputs

In [ ]:
A =  state

In [ ]:
A/255.

In [ ]:
in_data

In [ ]:
state.shape

In [ ]:
A.shape

In [ ]:
epochs = 300
min_batch_size = 32
inp_tensor = state
ep_list = []
loss_list = []
for epoch in range(epochs):
    step = 0
    scenario.reset(ego_vehicle)
    c = world.tick()
    done = False
    total_r = 0
    val = 0
    
    while not done:
        in_data = input_data.reshape((1,3,320,320))
        with torch.no_grad():
            st = torch.FloatTensor(in_data)
        ego_vehicle.apply_control(carla.VehicleControl(throttle=0.5))
        try:
            cmd, reward, done, _ = scenario.step(ego_vehicle)
        except:
            break
        c = world.tick()
        inp_tensor = torch.cat((inp_tensor,st),0)
        del st
        if inp_tensor.shape[0] >= min_batch_size:
            break
        step += 1
    if inp_tensor.shape[0] >= min_batch_size:
            optimizer.zero_grad()
            A = inp_tensor/255.
            #A -= A.min(1, keepdim=True)[0]
            #A /= A.max(1, keepdim=True)[0]
            inp_tensor = A
            outputs = ae(inp_tensor.to(device))
            loss = criterion(outputs, inp_tensor.to(device))
            loss.backward()
            optimizer.step()
            print("EPOCH:",epoch,"LOSS:",loss.item())
            ep_list.append(epoch)
            loss_list.append(loss.item())
            del inp_tensor,outputs,A
            inp_tensor = state

In [ ]:
BCE_loss = [ep_list,loss_list]

In [ ]:
MSE_loss_1 = [ep_list,loss_list]

In [ ]:
import pickle
f = open("MSE1.pkl",'wb')
pickle.dump(MSE_loss_1,f)
f.close()

In [ ]:
#plt.ylim(0,1)
plt.plot(ep_list,loss_list)

In [ ]:
in_data = state.detach().cpu().numpy()

In [ ]:
i_img = in_data.reshape((3,320,320)).reshape((320,320,3))

In [ ]:
plt.imshow(input_data)

In [ ]:
plt.imshow(i_img)

In [ ]:
state

In [ ]:
out =ae(state.to(device))

In [ ]:
scenario._target_lane_waypoint.transform.location.y

In [ ]:
ego_vehicle.get_location().y

In [ ]:
r = scenario.step(ego_vehicle)

In [ ]:
r

In [ ]:
oupp = out.detach().cpu().numpy()

In [ ]:
o_img = oupp.reshape((3,320,320)).reshape((320,320,3))

In [ ]:
plt.imshow(o_img)

In [ ]:
k.shape

In [57]:
total_reward_list = []
epoch_list = []
step_list = []

In [58]:
torch.cuda.empty_cache()

In [ ]:
import torchvision.models as models
mobilenet = models.mobilenet_v2(pretrained=True).to(device)

In [ ]:
del mobilenet

In [ ]:
mobilenet.classifier

In [ ]:
mobilenet.features

In [ ]:
in_data = a

In [ ]:
in_data  =in_data/255.

In [ ]:
with torch.no_grad():
    in_data = torch.FloatTensor(in_data).to(device)
    in_data = mobilenet.features(in_data)
    in_data = nn.AvgPool2d(7,7)(in_data)
    in_data = nn.Flatten()(in_data)
    in_data = in_data.detach().cpu().numpy()

In [ ]:
in_data.shape

In [ ]:
inps = np.array([0,0,1])

In [ ]:
in_data[:,]

In [ ]:
ap = np.concatenate((in_data[0],inps)).reshape(1,1283)

In [ ]:
with torch.no_grad():
    st = torch.FloatTensor(ap).to(device)

In [ ]:
st[:,-3:]

In [ ]:
f = in_data.reshape(128,100,3)

In [ ]:
plt.imshow(f)

In [ ]:
p.select_action(in_data)

In [ ]:
def PID_Controller(cmd,e_v,steer):
    speed_limit = 60
    min_speed = 10
    v = e_v.get_velocity()
    kmh = int(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2))
    if kmh > speed_limit:
        throttle = 0.1 + + random.random()/8
        brake = 0.9
    elif kmh < min_speed:
        throttle = 0.4 + random.random()/8
        brake = 0.0
    else:
        throttle = 0.1 + random.random()/8
        brake = 0
    
    if (val == 0  or val ==1):
        #Go Straight
        
        if abs(steer)> 0.1:
            
            throttle = 0.1 +  random.random()/8
            
                
            brake = (kmh-10)/50
            
        else:
            throttle = (60-kmh)/50 + random.random()/8
            brake = 0
            
        
        #s_clip_n = -0.15
        #s_clip_p = 0.15
        #t_clip_n = 0.4
        #t_clip_p = 1.0

    if (val == 2 or val == 5):
        # Go right
        
        if steer < 0:
            brake = (kmh-10)/50
            throttle = 0.1
        if steer > 0:
            throttle = (60-kmh)/50 + random.random()/8
            brake = 0
        #s_clip_n = 0.25
        #s_clip_p = 0.6
        #t_clip_n = 0.0
        #t_clip_p = 0.4

    if (val == 3 or val == 4):
        # Go Left
        if steer < 0 :
            throttle = (60-kmh)/50 + random.random()/8
            brake = 0
        if steer > 0 :
            brake = (kmh-10)/50
            throttle = 0.1
        #s_clip_n = -0.6
        #s_clip_p = -0.25
        #t_clip_n = 0.0
        #t_clip_p = 0.4
    if throttle <= 0:
        throttle = 0.05 + random.random()/50
    if throttle >= 0.4:
        throttle = 0.4 + random.random()/8
    if brake < 0:
        brake = 0
    if brake >= 1:
        brake = 0.9
    return throttle,brake

In [ ]:
import random
random.random()

In [ ]:
ego_vehicle.get_transform()

In [59]:
def way_cal(ego_vehicle,val,scenario = None):
    way = ego_vehicle.get_transform()
    #speed = scenario._veh.speed
    if scenario != None:
        way = scenario._target_lane_waypoint.transform
        way.location.x = scenario._veh.transform.as_carla_transform().location.x
    if (val == 0  or val ==1):
        way.location.x += 100       
    if (val == 2 or val == 5):
        way.location.x += 100
        #way.location.y += 5
    if (val == 3 or val == 4):
        way.location.x += 100
        #way.location.y -= 5
    if ego_vehicle.get_transform().rotation.yaw < -35:
        speed = 10
        way.location.y += 10
    if ego_vehicle.get_transform().rotation.yaw > 35:
        speed = 10
        way.location.y -= 10
    return way

In [60]:
def check_safety(way):
    d = scenario._ngsim_vehicles_in_carla._vehicle_by_vehicle_id
    y_left = way.location.y - 3.5
    y_right = way.location.y + 3.5
    x_lower = ego_vehicle.get_location().x + 5
    x_upper = ego_vehicle.get_location().x + 25
    
    car_in_zone = False
    for i in d:
        if d[i].get_location().x > x_lower and d[i].get_location().x < x_upper and d[i].get_location().y > y_left and d[i].get_location().y < y_right:
            car_in_zone = True
    #print(car_in_zone)
    safe = not car_in_zone
    #print(safe,"LOL")
    return safe


In [ ]:
ego_vehicle.get_location().y

In [ ]:
scenario._target_lane_waypoint.transform.location.y

In [ ]:
c = torch.arange(10).reshape(1,10)

In [ ]:
c = c.detach().numpy()

In [ ]:
d = torch.arange(4).reshape(4).detach().numpy()

In [ ]:
d

In [ ]:
c[0]

In [ ]:
d

In [ ]:
np.concatenate((c,d))

In [ ]:
torch.cat()

In [ ]:
scenario = prepare_ngsim_scenario(client)
world = client.get_world()
ego_vehicle = prepare_ego_vehicle(world)
scenario.reset(ego_vehicle)
world.tick()

In [ ]:
scenario.reset(ego_vehicle)
world.tick()

In [ ]:
len(list(world.get_actors()))

In [ ]:
ego_vehicle

In [ ]:
scenario.step(ego_vehicle)

In [ ]:
world.tick()

In [ ]:
ego_vehicle.apply_control(carla.VehicleControl(throttle=throttle, steer=np.clip(steer,-1.0,1.0),brake=brake))

In [ ]:
ego_vehicle

In [ ]:
for i in range(10):
    done = False
    
    scenario.reset(ego_vehicle)
    world.tick()
    #world = client.get_world()
    #spectator = world.get_spectator()
    #ego_vehicle = prepare_ego_vehicle(world)
    #scenario.reset(ego_vehicle)
    #world.tick()
    while not done:
        ego_vehicle.apply_control(carla.VehicleControl(throttle=throttle, steer=np.clip(steer,-1.0,1.0),brake=brake))
        r = scenario.step(ego_vehicle)
        print(r)
        done = r.done
        world.tick()
    print("RESET")
    for i in range(5):
        print()

In [ ]:
print(throttle,steer,brake)

In [ ]:
for i in list(world.get_actors()):
    if type(i) != carla.libcarla.Vehicle:
        print(i)
type(list(world.get_actors())[2]) == carla.libcarla.Vehicle

In [ ]:
s = os.environ.get("epseed")

In [ ]:
if s:
    print("HI")

In [ ]:
scenario

In [ ]:
len(total_reward_list)

In [ ]:
target_lane = scenario._lane_change.lane_to

In [ ]:
target_lane

In [61]:
def find_all():
    target_lane = scenario._lane_change.lane_to
    count = 0 
    for i in l:
        df = i._df
        df = df[df["Frame ID"] == scenario._lane_change.frame_start]
        lane_id = df["Lane Identification"].iloc[0]
        vehicle_laneID[i.id] = [lane_id,count]
        count+=1
    vehicles_in_lane = []
    vehicle_index = []
    for i in vehicle_laneID:
    if vehicle_laneID[i][0] == target_lane:
        vehicles_in_lane.append(i)
        vehicle_index.append(vehicle_laneID[i][1])

IndentationError: expected an indented block (<ipython-input-61-59d8328a52b9>, line 13)

In [62]:

def check_idx():
    d = scenario._ngsim_vehicles_in_carla._vehicle_by_vehicle_id
    not_ego = list(d.keys())
    ks  = scenario._ngsim_recording.env_cars
    for i in range(len(ks)):
        if ks[i].id not in not_ego:
            return i
    return -1


In [63]:
check_idx()

11

In [65]:
section = 0

In [64]:
mn=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

In [ ]:
way.location.x

In [ ]:
action[1]

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
epochs = 211
freq = 5
freq_n = 3
update_freq = 20
decay_freq = 1000
decay_c = 1000
#val = 0
min_r_avg = -1
steer_w = 1
min_steer_w = 0
steer_decay = 0.1
mn=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
last_epoch = -1

victory = 0
victory_buffer = 0
for epoch in range(0,epochs):
    step = 0
    
    #scenario = prepare_ngsim_scenario(client)
    #world = client.get_world()
    #ego_vehicle = prepare_ego_vehicle(world)
    #scenario.reset(ego_vehicle)
    #c = world.tick()
    #print(len(list(world.get_actors())))
    #del ego_vehicle
    ids = list(world.get_actors())[0]
    carla.command.DestroyActor(ids)
    '''
    for i in list(world.get_actors()):
        if type(i) != carla.libcarla.Vehicle:
            ids = i.id
            carla.command.DestroyActor(ids)
    '''
    world = client.get_world()
    #spectator = world.get_spectator()
    ego_vehicle = prepare_ego_vehicle(world)
    
    #scenario.close()
    #scenario = prepare_ngsim_scenario(client)
    
    #time.sleep(3)
    #world = client.get_world()
    #spectator = world.get_spectator()
    #client.reload_world()
    
    #ego_vehicle = prepare_ego_vehicle(world)
    
    scenario.reset(ego_vehicle)
    c = world.tick()
    
    ego_vehicle.apply_control(carla.VehicleControl(manual_gear_shift=True, gear=4))
    #ego_vehicle.apply_control(carla.VehicleControl(throttle=1.0, brake=1.0))
    c = world.tick()
    #ego_vehicle.apply_control(carla.VehicleControl(manual_gear_shift=False))
    #time.sleep(4)
    #ego_vehicle.apply_control(carla.VehicleControl(brake=0.0))
    c = world.tick()
    
    #way = scenario._target_lane_waypoint.transform
    speed = scenario._veh.speed
    #c = world.tick()
    done = False
    total_r = 0
    val = 0
    way = scenario._target_lane_waypoint.transform
    target_boundary = scenario._target_lane_waypoint.transform
    ##left_y = target_boundary.location.y - 5
    #right_y = target_boundary.location.y + 5
    #initial_x 
    #way = way_cal(ego_vehicle,val)
    #print(scenario._sampled_dataset_excerpt_info)
    t_clip_n = 0.0
    t_clip_p = 1.0
    
    s_clip_n = -1.0
    s_clip_p = 1.0
    cmd_buffer = [0]
    yaw_buffer = [0]
    
    steer = 0
    steer_aug = 0.1
    prep_limit = 10*3
    pid = VehiclePIDController(ego_vehicle,0.7)
    lock = 0
    if victory_buffer > 20:
        section+=1
        victory_buffer = 0
    while not done:
        '''
        while True:
            #print(current_frame,c)
            if current_frame >= c:
                #print(current_frame,c)
                break
        '''
        speed = scenario._veh.speed/1.603
        way = scenario._target_lane_waypoint.transform
        way.location.x = ego_vehicle.get_location().x + 30
        safe = check_safety(way)
        #print(safe)
        if step < prep_limit:
            k = pid.run_step(speed,way)
            #print(k.throttle)
            ego_vehicle.apply_control(carla.VehicleControl(throttle=k.throttle, steer=np.clip(0.0,-1.0,1.0),brake=k.brake))
        else:
            if lock == 0:
                pid = VehiclePIDController(ego_vehicle,0.7)
                lock = 1
            birdview = birdview_producer.produce(
                agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
                )
            a = birdview[0].reshape(1,50,150)
            a = np.append(a,birdview[1].reshape(1,50,150),axis=0)
            a = np.append(a,birdview[2].reshape(1,50,150),axis=0)
            a = np.append(a,birdview[3].reshape(1,50,150),axis=0)
            a = np.append(a,birdview[4].reshape(1,50,150),axis=0)
            #a = np.append(a,birdview[4].reshape(1,224,224),axis=0)
            #rgb = BirdViewProducer.as_rgb(birdview)/255.
            in_data = a.reshape(1,5,50,150)
            '''
            for i in range(3):
                rgb[:,:,i] = (rgb[:,:,i] - mn[i])/std[i]
            '''
            #in_data = rgb.reshape(1,3,224,224)
            '''
            with torch.no_grad():
                in_data = torch.FloatTensor(in_data).to(device)
                in_data = mobilenet.features(in_data)
                in_data = nn.AvgPool2d(7,7)(in_data)
                in_data = nn.Flatten()(in_data)
                in_data = in_data.detach().cpu().numpy()
            '''
            
            #in_data = mdl(in_data).numpy()
            #in_data = in_data.reshape(in_data.shape[0],62720)
            #in_data = in_data.reshape((1,5,186,150))
            #in_data = input_data.reshape((1,3,320,320))
            #print(in_data.shape)
            if val == 0 or val == 1:
                inps= [0,1,0]
            if val == 2 or val == 5:
                inps = [0,0,1]
            if val == 4 or val ==3:
                inps = [1,0,0]
            #inps = np.array(inps)
            inputs = [in_data,np.array(inps).reshape(1,3)]
            #ap = np.concatenate((in_data[0],inps)).reshape(1,1283)

            action = p.select_action(inputs)
            #print(action)
            #print(action)
            #action[0] = action[0]*2 - 1
            #action[1] = action[1]*120
            '''
            if (val == 0  or val ==1):
                s_clip_n = -0.15
                s_clip_p = 0.15
                t_clip_n = 0.4
                t_clip_p = 1.0

            if (val == 2 or val == 5):
                s_clip_n = 0.25
                s_clip_p = 0.8
                t_clip_n = 0.0
                t_clip_p = 0.4

            if (val == 3 or val == 4):
                s_clip_n = -0.8
                s_clip_p = -0.25
                t_clip_n = 0.0
                t_clip_p = 0.4
            '''


            #t_clip_n = 0.0
            #t_clip_p = 0.7

            #s_clip_n = -0.6
            #s_clip_p = 0.6  
            #d_action = action_space[action]
            #steer_s = action[0]*0.1
            #steer += steer_s
            '''
            if action[0]<0:
                steer -= steer_aug
            elif action[0]>0:
                steer += steer_aug
            
            if steer < -0.1:
                steer = -0.1
            if steer > 0.1:
                steer = 0.1
            '''
            #speed = action[0]*80+20
            #speed = action[1]
            #speed = math.sqrt(speed**2)
            #_speed = np.clip(action[0], -1,1)
            #speed = ((_speed + 1)/2)*50 + 10
            brake = 0
            throttle = 0
            steer = 0
            x = (action[0] + 1)/2
            x = x*120
            y = (action[1] + 1)/2
            y = y*130
            way.location.x =  ego_vehicle.get_location().x + x
            #way.location.y = ego_vehicle.get_location().y + y
            speed = y
            '''
            brake = 0
            throttle = 0
            if action[0] <0:
                brake = action[0]
                throttle = 0
            else:
                throttle = action[0]
                brake = 0
            '''
            '''
            if epoch > 400:
                if (val == 0  or val ==1):
                    s_clip_n = -0.15
                    s_clip_p = 0.15
                    t_clip_n = 0.4
                    t_clip_p = 1.0

                if (val == 2 or val == 5):
                    s_clip_n = 0.25
                    s_clip_p = 0.6
                    t_clip_n = 0.0
                    t_clip_p = 0.4

                if (val == 3 or val == 4):
                    s_clip_n = -0.6
                    s_clip_p = -0.25
                    t_clip_n = 0.0
                    t_clip_p = 0.4
            '''
            #if epoch < 20:
            #print(throttle,steer,brake)
            #pid = VehiclePIDController(ego_vehicle)

            k = pid.run_step(speed,way)
            #throttle = k.throttle
            #brake = k.brake

            avg_steer = k.steer 
            brake = k.brake
            throttle = k.throttle
            #print("SteerDiff:",abs(steer-k.steer),k.steer,steer)
            #print(throttle,steer,brake)
            #avg_steer= k.steer
            ego_vehicle.apply_control(carla.VehicleControl(throttle=throttle, steer=np.clip(avg_steer,-1.0,1.0),brake=brake))

        #ego_vehicle.apply_control(carla.VehicleControl(throttle=np.clip(throttle, t_clip_n, t_clip_p), steer=np.clip(action[1], s_clip_n, s_clip_p)))#,brake=np.clip(brake, 0.0, 1.0)))
        
        
        #print(steer,speed)
        try:
            cmd, reward, done, _ = scenario.step(ego_vehicle)
        except Exception as e:
            print(e)
            print("ERROR")
            reward = -1
            done = True
        #reward = reward/2
        
        '''
        if reward < 0 :
            reward = -0.1
        '''
        
        val = cmd.value
        #way = way_cal(ego_vehicle,val)
        cmd_buffer.append(val)
        yaw_buffer.append(ego_vehicle.get_transform().rotation.yaw)
        if section == 0:
            if len(cmd_buffer) > 10:
                if _['on_target_lane']:
                    reward = 1 
                    done = True
        if section == 1:
            if len(cmd_buffer) > 10:
                if sum(cmd_buffer[-10:]) == 0 and _['on_target_lane'] and total_r>=0.5:
                    reward = 1 
                    done = True
        if section == 2:
            if len(cmd_buffer) > 10:
                if sum(cmd_buffer[-10:]) == 0 and _['on_target_lane'] and sum([abs(x) for x in yaw_buffer[-10:]])/10<=10 and total_r>=0.5:
                    reward = 1 
                    done = True
        if section==3:
            if len(cmd_buffer) > 10:
                if sum(cmd_buffer[-10:]) == 0 and _['on_target_lane'] and sum([abs(x) for x in yaw_buffer[-10:]])/10<=10 and total_r>=0.6:
                    reward = 1 
                    done = True
        if section>=4:
            if len(cmd_buffer) > 10:
                if sum(cmd_buffer[-10:]) == 0 and _['on_target_lane'] and sum([abs(x) for x in yaw_buffer[-10:]])/10<=5 and total_r>=0.8:
                    reward = 1 
                    done = True
        
        if total_r >=0.9:
            print("Close to win")
        if reward >= 1:
            victory_buffer+=1
            print("Its a win")
            
        
            #p.save("WinModel"+str(epoch)+".mdl")
        #way = _["scenario_data"]["original_veh_transform"]
        #way = scenario._target_lane_waypoint.transform  
        
        
        #print(action[0],action[1])
        #print(done)
        #if done:
        #    print(_)
        #print(_)
        '''
        if (val == 0  or val ==1):
            if action[0] > 0.0:
                reward += 0.01
            if action[1] < 0.1 and action[1]> -0.1:
                reward += 0.01
                

        if (val == 2 or val == 5):
            if action[0] <0.1:
                reward += 0.01
            if action[1] > 0.0:
                reward += 0.01

        if (val == 3 or val == 4):
            if action[0] < 0.1:
                reward += 0.01
            if action[1] < 0.0:
                reward += 0.01
        '''
        #reward += step
        '''
        v = ego_vehicle.get_velocity()
        kmh = int(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2))
        
        if kmh < 60 & kmh > 0.2:
            #done = False
            reward += 1 #-1
            # Reward lighter steering when moving
            if np.abs(action[1]) < 0.3:
                reward += 1
            elif np.abs(action[1]) > 0.5 and np.abs(action[1]) < 0.9:
                reward -= 0.1
            elif np.abs(action[1]) >= 0.9:
                reward -= 0.2
        elif kmh < 0.2:
            reward -= 0.1
        else:
            #print("Maybe never")
            reward += 0.01
            if np.abs(action[1]) < 0.3:
                reward += 0.12
            # Reduce score for heavy steering
            if np.abs(action[1]) > 0.5 and np.abs(action[1]) < 0.9:
                reward -= 0.17
            elif np.abs(action[1]) >= 0.9:
                reward -= 0.21
        '''
        '''
        rgb = BirdViewProducer.as_rgb(birdview)
        cv2.imshow('Frame',rgb)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
        '''
        if step < prep_limit:
            pass
        else:
            #print("HI")
            p.buffer.rewards.append(reward)
            p.buffer.is_terminals.append(done)
        
        total_r += reward
        step += 1
        
        '''
        if step % freq ==0 :
            print(step)
            p.update()
        
        if step % freq_n == 0:
            p.decay_action_std(0.05,0.1)
        
        
        '''
        world.tick()
    #total_r += step/10
    '''
    if epoch > 50:
        
        if sum(total_reward_list[-50:])/len(total_reward_list[-50:]) > min_r_avg or sum(step_list) > decay_freq:
            decay_freq += decay_c
            #decay_c -= 50
            #print("Decaying:",p.action_std)
            p.decay_action_std(0.01,0.1)
            
            min_r_avg = sum(total_reward_list[-50:])/len(total_reward_list[-50:])
    '''
    if epoch%update_freq ==0 and epoch != 0 and len(p.buffer.states)>1:
            print(k.steer,speed)
            print("Update with batches:",len(p.buffer.states))
            p.update()
            #update_freq = 10
            if steer_w > min_steer_w +0.1:
                steer_w -= steer_decay
    '''
    else:
            print(total_r,epoch,step)
            total_reward_list.append(total_r)
            epoch_list.append(epoch)
            step_list.append(step)
            continue
    #except Exception as e:
        #print("Error:",e)
        #pass
    '''
    if step <10 and epoch != 0 and epoch!=last_epoch+1:
        last_epoch = epoch
        scenario = prepare_ngsim_scenario(client)
        world = client.get_world()
        ego_vehicle = prepare_ego_vehicle(world)
        scenario.reset(ego_vehicle)
        world.tick()
    cv2.destroyAllWindows()
    print(total_r,epoch,step,"Section:",section,"V_Buffer:",victory_buffer)
    total_reward_list.append(total_r)
    epoch_list.append(epoch)
    step_list.append(step)
    

Its a win
1.7 0 10 Section: 0 V_Buffer: 1
-0.5 1 54 Section: 0 V_Buffer: 1
carla_real_traffic_scenarios.ngsim.ngsim_recording.NGSimCar.66 has undefined direction, assuming horizontal
Its a win
1.1 2 22 Section: 0 V_Buffer: 2


Switch synchronous_mode=True
Change fixed_delta_seconds=0.1


-1.0 3 1 Section: 0 V_Buffer: 2
Its a win
1.1 4 40 Section: 0 V_Buffer: 3
Its a win
0.8 5 38 Section: 0 V_Buffer: 4
Its a win
1.6 6 83 Section: 0 V_Buffer: 5
Its a win
1.5 7 51 Section: 0 V_Buffer: 6
Its a win
1.2 8 41 Section: 0 V_Buffer: 7
Its a win
1.1 9 50 Section: 0 V_Buffer: 8
Its a win
1.6 10 66 Section: 0 V_Buffer: 9
Its a win
1.3 11 53 Section: 0 V_Buffer: 10
Its a win
1.3 12 36 Section: 0 V_Buffer: 11
Its a win
1.0 13 46 Section: 0 V_Buffer: 12
Its a win
1.6 14 73 Section: 0 V_Buffer: 13
Its a win
1.1 15 10 Section: 0 V_Buffer: 14
Its a win
1.5 16 67 Section: 0 V_Buffer: 15
Its a win
1.4 17 57 Section: 0 V_Buffer: 16
Its a win
1.1 18 25 Section: 0 V_Buffer: 17
Its a win
1.1 19 45 Section: 0 V_Buffer: 18
-0.07385468482971191 34.2340672537489
Update with batches: 350
torch.Size([350, 5, 50, 150]) torch.Size([350, 2]) torch.Size([350, 2])
-1.0 20 10 Section: 0 V_Buffer: 18
Its a win
1.5 21 59 Section: 0 V_Buffer: 19
Its a win
1.6 22 61 Section: 0 V_Buffer: 20
Its a win
1.5 23 46

Switch synchronous_mode=True
Change fixed_delta_seconds=0.1


-1.0 40 3 Section: 1 V_Buffer: 12
Close to win
Close to win
Close to win
Close to win
Close to win
Its a win
1.9 41 47 Section: 1 V_Buffer: 13
Its a win
1.5 42 64 Section: 1 V_Buffer: 14
-0.8 43 42 Section: 1 V_Buffer: 14
Its a win
1.5 44 62 Section: 1 V_Buffer: 15
-0.4 45 54 Section: 1 V_Buffer: 15
-0.7 46 45 Section: 1 V_Buffer: 15
-1.0 47 16 Section: 1 V_Buffer: 15
Its a win
1.5 48 52 Section: 1 V_Buffer: 16
Close to win
Close to win
-0.09999999999999998 49 44 Section: 1 V_Buffer: 16
Its a win
1.6 50 78 Section: 1 V_Buffer: 17


...

-1.0 65 1 Section: 2 V_Buffer: 0
-1.0 66 11 Section: 2 V_Buffer: 0


Switch synchronous_mode=True
Change fixed_delta_seconds=0.1


-1.0 67 3 Section: 2 V_Buffer: 0
Close to win
Close to win
Close to win
Close to win
Close to win
Close to win
Close to win
Its a win
1.9 68 53 Section: 2 V_Buffer: 1
Its a win
1.5 69 43 Section: 2 V_Buffer: 2
Close to win
-0.5 70 46 Section: 2 V_Buffer: 2
Its a win
1.5 71 38 Section: 2 V_Buffer: 3
Close to win
-0.4 72 46 Section: 2 V_Buffer: 3
Its a win
1.5 73 63 Section: 2 V_Buffer: 4
Its a win
1.7 74 63 Section: 2 V_Buffer: 5
Its a win
1.6 75 38 Section: 2 V_Buffer: 6
-0.6 76 44 Section: 2 V_Buffer: 6
Its a win
1.6 77 75 Section: 2 V_Buffer: 7
Its a win
1.6 78 50 Section: 2 V_Buffer: 8


Switch synchronous_mode=True
Change fixed_delta_seconds=0.1


-1.0 79 1 Section: 2 V_Buffer: 8
0.07414429634809494 72.40335315465927
Update with batches: 293
torch.Size([293, 5, 50, 150]) torch.Size([293, 2]) torch.Size([293, 2])
-1.0 80 39 Section: 2 V_Buffer: 8
Its a win
1.5 81 50 Section: 2 V_Buffer: 9
Close to win
Close to win
Close to win
Close to win
Its a win
1.6 82 49 Section: 2 V_Buffer: 10
Its a win
1.8 83 55 Section: 2 V_Buffer: 11
Its a win
1.8 84 72 Section: 2 V_Buffer: 12
Its a win
1.5 85 65 Section: 2 V_Buffer: 13
Close to win
-0.30000000000000004 86 45 Section: 2 V_Buffer: 13
Its a win
1.6 87 60 Section: 2 V_Buffer: 14
Close to win
Close to win
-0.5 88 56 Section: 2 V_Buffer: 14
Its a win
1.8 89 15 Section: 2 V_Buffer: 15
Its a win
1.6 90 73 Section: 2 V_Buffer: 16
Close to win
Close to win
Close to win
Close to win
Close to win
Close to win
Close to win
Close to win
Close to win
Its a win
1.9 91 53 Section: 2 V_Buffer: 17
Close to win
-0.4 92 44 Section: 2 V_Buffer: 17
Its a win
1.8 93 70 Section: 2 V_Buffer: 18
Close to win
Clos

In [ ]:
section

In [ ]:
p.buffer.clear()

In [ ]:
p.save("Latest.mdl")

In [ ]:
p.load("Latest.mdl")

In [ ]:
step = 0
tr_x = []
tr_y = []
e_x = []
e_y = []
spd = []
spd2 = []
#scenario = prepare_ngsim_scenario(client)
#world = client.get_world()
#ego_vehicle = prepare_ego_vehicle(world)
#scenario.reset(ego_vehicle)
#c = world.tick()
#print(len(list(world.get_actors())))
#del ego_vehicle
ids = list(world.get_actors())[0]
carla.command.DestroyActor(ids)
'''
for i in list(world.get_actors()):
    if type(i) != carla.libcarla.Vehicle:
        ids = i.id
        carla.command.DestroyActor(ids)
'''
world = client.get_world()
#spectator = world.get_spectator()
ego_vehicle = prepare_ego_vehicle(world)

#scenario.close()
#scenario = prepare_ngsim_scenario(client)

#time.sleep(3)
#world = client.get_world()
#spectator = world.get_spectator()
#client.reload_world()

#ego_vehicle = prepare_ego_vehicle(world)

scenario.reset(ego_vehicle)
c = world.tick()
c = world.tick()
ego_vehicle.apply_control(carla.VehicleControl(manual_gear_shift=True, gear=4))
c = world.tick()

#way = scenario._target_lane_waypoint.transform
speed = scenario._veh.speed
#c = world.tick()
done = False
total_r = 0
val = 0
way = scenario._target_lane_waypoint.transform
#way = way_cal(ego_vehicle,val)
#print(scenario._sampled_dataset_excerpt_info)
t_clip_n = 0.0
t_clip_p = 1.0

s_clip_n = -1.0
s_clip_p = 1.0
cmd_buffer = [0]
yaw_buffer = [0]
steer = 0
steer_aug = 0.1
prep_limit = 10*1
pid = VehiclePIDController(ego_vehicle,0.7)
lock = 0
while not done:
    '''
    while True:
        #print(current_frame,c)
        if current_frame >= c:
            #print(current_frame,c)
            break
    '''
    speed = scenario._veh.speed/1.603
    way = scenario._target_lane_waypoint.transform
    way.location.x = ego_vehicle.get_location().x + 30
    safe = check_safety(way)
    #print(safe)
    #print(speed)
    ids = check_idx()
    actions = scenario._ngsim_recording.env_cars[ids].policy()
    scenario._ngsim_recording.env_cars[ids].step(actions)
    #actions = actions/0.1
    #print(actions)
    if step < prep_limit:
        k = pid.run_step(speed,way)
        #print(k.throttle)
        
        ego_vehicle.apply_control(carla.VehicleControl(throttle=k.throttle, steer=np.clip(0.0,-1.0,1.0),brake=k.brake))
    else:
        if lock == 0:
            pid = VehiclePIDController(ego_vehicle,0.7)
            lock = 1
        birdview = birdview_producer.produce(
            agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
            )
        a = birdview[0].reshape(1,186,150)
        a = np.append(a,birdview[1].reshape(1,186,150),axis=0)
        a = np.append(a,birdview[2].reshape(1,186,150),axis=0)
        a = np.append(a,birdview[3].reshape(1,186,150),axis=0)
        a = np.append(a,birdview[4].reshape(1,186,150),axis=0)
        #a = np.append(a,birdview[4].reshape(1,224,224),axis=0)
        #rgb = BirdViewProducer.as_rgb(birdview)/255.
        in_data = a.reshape(1,5,186,150)
        #a = birdview[0].reshape(1,186,150)
        #a = np.append(a,birdview[1].reshape(1,186,150),axis=0)
        #a = np.append(a,birdview[2].reshape(1,186,150),axis=0)
        #a = np.append(a,birdview[3].reshape(1,186,150),axis=0)
        #a = np.append(a,birdview[4].reshape(1,186,150),axis=0)
        #a = np.append(a,birdview[4].reshape(1,224,224),axis=0)
        '''
        rgb = BirdViewProducer.as_rgb(birdview)/255.
        #in_data = a.reshape(1,5,186,150)
        for i in range(3):
            rgb[:,:,i] = (rgb[:,:,i] - mn[i])/std[i]
        in_data = rgb.reshape(1,3,224,224)

        with torch.no_grad():
            in_data = torch.FloatTensor(in_data).to(device)
            in_data = mobilenet.features(in_data)
            in_data = nn.AvgPool2d(7,7)(in_data)
            in_data = nn.Flatten()(in_data)
            in_data = in_data.detach().cpu().numpy()
        #in_data = mdl(in_data).numpy()
        #in_data = in_data.reshape(in_data.shape[0],62720)
        #in_data = in_data.reshape((1,5,186,150))
        #in_data = input_data.reshape((1,3,320,320))
        #print(in_data.shape)
        '''
        if val == 0 or val == 1:
            inps= [0,1,0]
        if val == 2 or val == 5:
            inps = [0,0,1]
        if val == 4 or val ==3:
            inps = [1,0,0]
        #inps = np.array(inps)
        inputs = [in_data,np.array(inps).reshape(1,3)]
        #ap = np.concatenate((in_data[0],inps)).reshape(1,1283)

        action = p.select_action(inputs,True)
        #print(action)
        #print(action)
        #action[0] = action[0]*2 - 1
        #action[1] = action[1]*120
        '''
        if (val == 0  or val ==1):
            s_clip_n = -0.15
            s_clip_p = 0.15
            t_clip_n = 0.4
            t_clip_p = 1.0

        if (val == 2 or val == 5):
            s_clip_n = 0.25
            s_clip_p = 0.8
            t_clip_n = 0.0
            t_clip_p = 0.4

        if (val == 3 or val == 4):
            s_clip_n = -0.8
            s_clip_p = -0.25
            t_clip_n = 0.0
            t_clip_p = 0.4
        '''


        #t_clip_n = 0.0
        #t_clip_p = 0.7

        #s_clip_n = -0.6
        #s_clip_p = 0.6  
        #d_action = action_space[action]
        #steer_s = action[0]*0.1
        #steer += steer_s
        steer = 0
        x = (action[0] + 1)/2
        x = x*100
        y = (action[1] + 1)/2
        y = y*130
        way.location.x =  ego_vehicle.get_location().x + x
        speed = y
        #way.location.y = ego_vehicle.get_location().y + y
        #way.location.x =  ego_vehicle.get_location().x + action[0]
        #way.location.y = ego_vehicle.get_location().y + action[1]
        '''
        if steer < -0.1:
            steer = -0.1
        if steer > 0.1:
            steer = 0.1
        '''
        #speed = action[0]*80+20
        #print(speed)
        #speed = action[1]
        #speed = math.sqrt(speed**2)
        #_speed = np.clip(action[0], -1,1)
        #speed = ((_speed + 1)/2)*50 + 10
        brake = 0
        throttle = 0
        '''
        brake = 0
        throttle = 0
        if action[0] <0:
            brake = action[0]
            throttle = 0
        else:
            throttle = action[0]
            brake = 0
        '''
        '''
        if epoch > 400:
            if (val == 0  or val ==1):
                s_clip_n = -0.15
                s_clip_p = 0.15
                t_clip_n = 0.4
                t_clip_p = 1.0

            if (val == 2 or val == 5):
                s_clip_n = 0.25
                s_clip_p = 0.6
                t_clip_n = 0.0
                t_clip_p = 0.4

            if (val == 3 or val == 4):
                s_clip_n = -0.6
                s_clip_p = -0.25
                t_clip_n = 0.0
                t_clip_p = 0.4
        '''
        #if epoch < 20:
        #print(throttle,steer,brake)
        #pid = VehiclePIDController(ego_vehicle)

        k = pid.run_step(speed,way)
        #throttle = k.throttle
        #brake = k.brake

        avg_steer = k.steer 
        brake = k.brake
        throttle = k.throttle
        #print("SteerDiff:",abs(steer-k.steer),k.steer,steer)
        #print(steer,k.steer)
        #avg_steer = k.steer
        '''
        if actions[0]>0:
            
            throttle = actions[0]
            brake = 0.0
        else:
            throttle = 0.0
            brake = -1*actions[0]
        '''
        #avg_steer = k.steer
        ego_vehicle.apply_control(carla.VehicleControl(throttle=throttle, steer=np.clip(avg_steer,-1.0,1.0),brake=brake))
        print(way.location.x,way.location.y,ego_vehicle.get_location().x,ego_vehicle.get_location().y)
        tr_x.append(x)
        tr_y.append(way.location.y)
        e_x.append(ego_vehicle.get_location().x)
        e_y.append(ego_vehicle.get_location().y)
        spd.append(speed)
        spd2.append(scenario._veh.speed/1.603)
        #ego_vehicle.apply_control(carla.VehicleControl(throttle=np.clip(throttle, t_clip_n, t_clip_p), steer=np.clip(action[1], s_clip_n, s_clip_p)))#,brake=np.clip(brake, 0.0, 1.0)))


    #print(steer,speed)
    try:
        cmd, reward, done, _ = scenario.step(ego_vehicle)
    except Exception as e:
        print(e)
        print("ERROR")
        reward = -1
        done = True


    '''
    if reward < 0 :
        reward = -0.1
    '''

    val = cmd.value
    #way = way_cal(ego_vehicle,val)
    cmd_buffer.append(val)
    yaw_buffer.append(ego_vehicle.get_transform().rotation.yaw)
    
    if len(cmd_buffer) > 10:
        if sum(cmd_buffer[-10:]) == 0 and _['on_target_lane'] and sum([abs(x) for x in yaw_buffer[-10:]])/10<=10 and total_r>=0.9:
            reward = 1
            done = True

    
    if total_r >=0.9:
        print("Close to win")
    if reward >= 1:
        print("Its a win")


        #p.save("WinModel"+str(epoch)+".mdl")
    #way = _["scenario_data"]["original_veh_transform"]
    #way = scenario._target_lane_waypoint.transform  


    #print(action[0],action[1])
    #print(done)
    #if done:
    #    print(_)
    #print(_)
    '''
    if (val == 0  or val ==1):
        if action[0] > 0.0:
            reward += 0.01
        if action[1] < 0.1 and action[1]> -0.1:
            reward += 0.01


    if (val == 2 or val == 5):
        if action[0] <0.1:
            reward += 0.01
        if action[1] > 0.0:
            reward += 0.01

    if (val == 3 or val == 4):
        if action[0] < 0.1:
            reward += 0.01
        if action[1] < 0.0:
            reward += 0.01
    '''
    #reward += step
    '''
    v = ego_vehicle.get_velocity()
    kmh = int(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2))

    if kmh < 60 & kmh > 0.2:
        #done = False
        reward += 1 #-1
        # Reward lighter steering when moving
        if np.abs(action[1]) < 0.3:
            reward += 1
        elif np.abs(action[1]) > 0.5 and np.abs(action[1]) < 0.9:
            reward -= 0.1
        elif np.abs(action[1]) >= 0.9:
            reward -= 0.2
    elif kmh < 0.2:
        reward -= 0.1
    else:
        #print("Maybe never")
        reward += 0.01
        if np.abs(action[1]) < 0.3:
            reward += 0.12
        # Reduce score for heavy steering
        if np.abs(action[1]) > 0.5 and np.abs(action[1]) < 0.9:
            reward -= 0.17
        elif np.abs(action[1]) >= 0.9:
            reward -= 0.21
    '''
    
    rgb = BirdViewProducer.as_rgb(birdview)
    cv2.imshow('Frame',rgb)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
    
    if step < prep_limit:
        pass
    else:
        #print("HI")
        p.buffer.rewards.append(reward)
        p.buffer.is_terminals.append(done)

    total_r += reward
    step += 1
    v = ego_vehicle.get_velocity()
    kmh = int(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2))
    #print(kmh,reward)
    '''
    if step % freq ==0 :
        print(step)
        p.update()

    if step % freq_n == 0:
        p.decay_action_std(0.05,0.1)


    '''
    world.tick()
#total_r += step/10
'''
if epoch > 50:

    if sum(total_reward_list[-50:])/len(total_reward_list[-50:]) > min_r_avg or sum(step_list) > decay_freq:
        decay_freq += decay_c
        #decay_c -= 50
        #print("Decaying:",p.action_std)
        p.decay_action_std(0.01,0.1)

        min_r_avg = sum(total_reward_list[-50:])/len(total_reward_list[-50:])
'''
'''
if epoch%update_freq ==0 and epoch != 0:
        print(steer)
        print("Update with batches:",len(p.buffer.states))
        p.update()
        if steer_w > min_steer_w +0.1:
            steer_w -= steer_decay
'''
'''
else:
        print(total_r,epoch,step)
        total_reward_list.append(total_r)
        epoch_list.append(epoch)
        step_list.append(step)
        continue
#except Exception as e:
    #print("Error:",e)
    #pass
'''
'''
#cv2.destroyAllWindows()
#print(total_r,epoch,step)
#total_reward_list.append(total_r)
#epoch_list.append(epoch)
#step_list.append(step)
'''
print("HI")

In [ ]:
spd,spd2

In [ ]:
cv2.destroyAllWindows()

In [ ]:
len(yaw_buffer[11:])

In [ ]:
yaw_x = yaw_buffer[11:]

In [ ]:
len(lcr_x)

In [ ]:
scenario.reset(ego_vehicle)
c = world.tick()
c = world.tick()
ego_vehicle.apply_control(carla.VehicleControl(manual_gear_shift=True, gear=4))
c = world.tick()

In [ ]:
ego_vehicle.get_location().x,ego_vehicle.get_location().y

In [ ]:
tr_x.append(ego_vehicle.get_location().x)
tr_y.append(ego_vehicle.get_location().y)

In [ ]:
n = list(range(len(tr_y)))

In [ ]:
n

In [ ]:
cr_x = (np.array(tr_x) - min(tr_x))/(max(tr_x) - min(tr_x))

In [ ]:
er_x = (np.array(e_y) - min(e_y))/(max(e_y) - min(e_y))

In [ ]:
n.append(1000)

In [ ]:
from matplotlib.animation import FuncAnimation

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
#plt.xlim(-10.748,-0.748)
#plt.ylim(min(tr_x)-0.1,max(tr_x)+0.1)
#ax.invert_xaxis()
#plt.ylim()
scat = ax.scatter([],[])

def anim(i):
    scat = ax.scatter(tr_y[:i+1],tr_x[:i+1])
#ax.scatter(e_y,e_x)
    for j, txt in enumerate(n[:i+1]):
            ax.annotate(txt, (tr_y[:i+1][j], tr_x[:i+1][j]))
    return scat,
anim = FuncAnimation(fig, anim, frames=len(n), interval=1000, repeat=False)
plt.show()

In [ ]:
plt.plot(n,list(cr_x))
plt.plot(n,list(er_x))

In [ ]:
dw = [0]
ew = [0]
lcr_x = list(cr_x)
ler_x = list(er_x)
for i in range(1,len(lcr_x)):
    dw.append(lcr_x[i] - lcr_x[i-1])
    ew.append(ler_x[i] - ler_x[i-1])
    

In [ ]:
[x*0.8 for x in ew]

In [ ]:
#plt.plot(n,lcr_x)
plt.plot(n,[x*100 for x in ew])
plt.plot(n,yaw_x)

In [ ]:
for i, txt in enumerate(n):
        ax.annotate(txt, (y[i], x[i]))

In [ ]:
plt.scatter(tr_y,tr_x)

In [ ]:
with torch.no_grad():
    print(p.policy.action_log_std.exp())

In [ ]:
action

In [ ]:
_

In [ ]:
cmd_buffer

In [ ]:
yaw_buffer

In [ ]:
total_r,step

In [ ]:
scenario._sampled_dataset_excerpt_info

In [ ]:
list

In [ ]:
cv2.destroyAllWindows()

In [ ]:
scenario._target_lane_waypoint.transform

In [ ]:
scenario._veh.speed

In [ ]:
val_score = 0
for ci in range(50):
    step = 0
    tr_x = []
    tr_y = []
    e_x = []
    e_y = []
    spd = []
    spd2 = []
    #scenario = prepare_ngsim_scenario(client)
    #world = client.get_world()
    #ego_vehicle = prepare_ego_vehicle(world)
    #scenario.reset(ego_vehicle)
    #c = world.tick()
    #print(len(list(world.get_actors())))
    #del ego_vehicle
    ids = list(world.get_actors())[0]
    carla.command.DestroyActor(ids)
    '''
    for i in list(world.get_actors()):
        if type(i) != carla.libcarla.Vehicle:
            ids = i.id
            carla.command.DestroyActor(ids)
    '''
    world = client.get_world()
    #spectator = world.get_spectator()
    ego_vehicle = prepare_ego_vehicle(world)

    #scenario.close()
    #scenario = prepare_ngsim_scenario(client)

    #time.sleep(3)
    #world = client.get_world()
    #spectator = world.get_spectator()
    #client.reload_world()

    #ego_vehicle = prepare_ego_vehicle(world)

    scenario.reset(ego_vehicle)
    c = world.tick()
    c = world.tick()
    ego_vehicle.apply_control(carla.VehicleControl(manual_gear_shift=True, gear=4))
    c = world.tick()

    #way = scenario._target_lane_waypoint.transform
    speed = scenario._veh.speed
    #c = world.tick()
    done = False
    total_r = 0
    val = 0
    way = scenario._target_lane_waypoint.transform
    #way = way_cal(ego_vehicle,val)
    #print(scenario._sampled_dataset_excerpt_info)
    t_clip_n = 0.0
    t_clip_p = 1.0

    s_clip_n = -1.0
    s_clip_p = 1.0
    cmd_buffer = [0]
    yaw_buffer = [0]
    steer = 0
    steer_aug = 0.1
    prep_limit = 10*1
    pid = VehiclePIDController(ego_vehicle,0.7)
    lock = 0
    while not done:
        '''
        while True:
            #print(current_frame,c)
            if current_frame >= c:
                #print(current_frame,c)
                break
        '''
        speed = scenario._veh.speed/1.603
        way = scenario._target_lane_waypoint.transform
        way.location.x = ego_vehicle.get_location().x + 30
        safe = check_safety(way)
        #print(safe)
        #print(speed)
        ids = check_idx()
        actions = scenario._ngsim_recording.env_cars[ids].policy()
        scenario._ngsim_recording.env_cars[ids].step(actions)
        #actions = actions/0.1
        #print(actions)
        if step < prep_limit:
            k = pid.run_step(speed,way)
            #print(k.throttle)

            ego_vehicle.apply_control(carla.VehicleControl(throttle=k.throttle, steer=np.clip(0.0,-1.0,1.0),brake=k.brake))
        else:
            if lock == 0:
                pid = VehiclePIDController(ego_vehicle,0.7)
                lock = 1
            birdview = birdview_producer.produce(
                agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
                )
            a = birdview[0].reshape(1,186,150)
            a = np.append(a,birdview[1].reshape(1,186,150),axis=0)
            a = np.append(a,birdview[2].reshape(1,186,150),axis=0)
            a = np.append(a,birdview[3].reshape(1,186,150),axis=0)
            a = np.append(a,birdview[4].reshape(1,186,150),axis=0)
            #a = np.append(a,birdview[4].reshape(1,224,224),axis=0)
            #rgb = BirdViewProducer.as_rgb(birdview)/255.
            in_data = a.reshape(1,5,186,150)
            #a = birdview[0].reshape(1,186,150)
            #a = np.append(a,birdview[1].reshape(1,186,150),axis=0)
            #a = np.append(a,birdview[2].reshape(1,186,150),axis=0)
            #a = np.append(a,birdview[3].reshape(1,186,150),axis=0)
            #a = np.append(a,birdview[4].reshape(1,186,150),axis=0)
            #a = np.append(a,birdview[4].reshape(1,224,224),axis=0)
            '''
            rgb = BirdViewProducer.as_rgb(birdview)/255.
            #in_data = a.reshape(1,5,186,150)
            for i in range(3):
                rgb[:,:,i] = (rgb[:,:,i] - mn[i])/std[i]
            in_data = rgb.reshape(1,3,224,224)

            with torch.no_grad():
                in_data = torch.FloatTensor(in_data).to(device)
                in_data = mobilenet.features(in_data)
                in_data = nn.AvgPool2d(7,7)(in_data)
                in_data = nn.Flatten()(in_data)
                in_data = in_data.detach().cpu().numpy()
            #in_data = mdl(in_data).numpy()
            #in_data = in_data.reshape(in_data.shape[0],62720)
            #in_data = in_data.reshape((1,5,186,150))
            #in_data = input_data.reshape((1,3,320,320))
            #print(in_data.shape)
            '''
            if val == 0 or val == 1:
                inps= [0,1,0]
            if val == 2 or val == 5:
                inps = [0,0,1]
            if val == 4 or val ==3:
                inps = [1,0,0]
            #inps = np.array(inps)
            inputs = [in_data,np.array(inps).reshape(1,3)]
            #ap = np.concatenate((in_data[0],inps)).reshape(1,1283)

            action = p.select_action(inputs,True)
            #print(action)
            #print(action)
            #action[0] = action[0]*2 - 1
            #action[1] = action[1]*120
            '''
            if (val == 0  or val ==1):
                s_clip_n = -0.15
                s_clip_p = 0.15
                t_clip_n = 0.4
                t_clip_p = 1.0

            if (val == 2 or val == 5):
                s_clip_n = 0.25
                s_clip_p = 0.8
                t_clip_n = 0.0
                t_clip_p = 0.4

            if (val == 3 or val == 4):
                s_clip_n = -0.8
                s_clip_p = -0.25
                t_clip_n = 0.0
                t_clip_p = 0.4
            '''


            #t_clip_n = 0.0
            #t_clip_p = 0.7

            #s_clip_n = -0.6
            #s_clip_p = 0.6  
            #d_action = action_space[action]
            #steer_s = action[0]*0.1
            #steer += steer_s
            steer = 0
            x = (action[0] + 1)/2
            x = x*100
            y = (action[1] + 1)/2
            y = y*130
            way.location.x =  ego_vehicle.get_location().x + x
            speed = y
            #way.location.y = ego_vehicle.get_location().y + y
            #way.location.x =  ego_vehicle.get_location().x + action[0]
            #way.location.y = ego_vehicle.get_location().y + action[1]
            '''
            if steer < -0.1:
                steer = -0.1
            if steer > 0.1:
                steer = 0.1
            '''
            #speed = action[0]*80+20
            #print(speed)
            #speed = action[1]
            #speed = math.sqrt(speed**2)
            #_speed = np.clip(action[0], -1,1)
            #speed = ((_speed + 1)/2)*50 + 10
            brake = 0
            throttle = 0
            '''
            brake = 0
            throttle = 0
            if action[0] <0:
                brake = action[0]
                throttle = 0
            else:
                throttle = action[0]
                brake = 0
            '''
            '''
            if epoch > 400:
                if (val == 0  or val ==1):
                    s_clip_n = -0.15
                    s_clip_p = 0.15
                    t_clip_n = 0.4
                    t_clip_p = 1.0

                if (val == 2 or val == 5):
                    s_clip_n = 0.25
                    s_clip_p = 0.6
                    t_clip_n = 0.0
                    t_clip_p = 0.4

                if (val == 3 or val == 4):
                    s_clip_n = -0.6
                    s_clip_p = -0.25
                    t_clip_n = 0.0
                    t_clip_p = 0.4
            '''
            #if epoch < 20:
            #print(throttle,steer,brake)
            #pid = VehiclePIDController(ego_vehicle)

            k = pid.run_step(speed,way)
            #throttle = k.throttle
            #brake = k.brake

            avg_steer = k.steer 
            brake = k.brake
            throttle = k.throttle
            #print("SteerDiff:",abs(steer-k.steer),k.steer,steer)
            #print(steer,k.steer)
            #avg_steer = k.steer
            '''
            if actions[0]>0:

                throttle = actions[0]
                brake = 0.0
            else:
                throttle = 0.0
                brake = -1*actions[0]
            '''
            #avg_steer = k.steer
            ego_vehicle.apply_control(carla.VehicleControl(throttle=throttle, steer=np.clip(avg_steer,-1.0,1.0),brake=brake))
            #print(way.location.x,way.location.y,ego_vehicle.get_location().x,ego_vehicle.get_location().y)
            tr_x.append(x)
            tr_y.append(way.location.y)
            e_x.append(ego_vehicle.get_location().x)
            e_y.append(ego_vehicle.get_location().y)
            spd.append(speed)
            spd2.append(scenario._veh.speed/1.603)
            #ego_vehicle.apply_control(carla.VehicleControl(throttle=np.clip(throttle, t_clip_n, t_clip_p), steer=np.clip(action[1], s_clip_n, s_clip_p)))#,brake=np.clip(brake, 0.0, 1.0)))


        #print(steer,speed)
        try:
            cmd, reward, done, _ = scenario.step(ego_vehicle)
        except Exception as e:
            print(e)
            print("ERROR")
            reward = -1
            done = True


        '''
        if reward < 0 :
            reward = -0.1
        '''

        val = cmd.value
        #way = way_cal(ego_vehicle,val)
        cmd_buffer.append(val)
        yaw_buffer.append(ego_vehicle.get_transform().rotation.yaw)

        if len(cmd_buffer) > 10:
            if sum(cmd_buffer[-10:]) == 0 and _['on_target_lane'] and sum([abs(x) for x in yaw_buffer[-10:]])/10<=10 and total_r>=0.9:
                reward = 1
                done = True


        if total_r >=0.9:
            print("Close to win")
        if reward >= 1:
            print("Its a win")


            #p.save("WinModel"+str(epoch)+".mdl")
        #way = _["scenario_data"]["original_veh_transform"]
        #way = scenario._target_lane_waypoint.transform  


        #print(action[0],action[1])
        #print(done)
        #if done:
        #    print(_)
        #print(_)
        '''
        if (val == 0  or val ==1):
            if action[0] > 0.0:
                reward += 0.01
            if action[1] < 0.1 and action[1]> -0.1:
                reward += 0.01


        if (val == 2 or val == 5):
            if action[0] <0.1:
                reward += 0.01
            if action[1] > 0.0:
                reward += 0.01

        if (val == 3 or val == 4):
            if action[0] < 0.1:
                reward += 0.01
            if action[1] < 0.0:
                reward += 0.01
        '''
        #reward += step
        '''
        v = ego_vehicle.get_velocity()
        kmh = int(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2))

        if kmh < 60 & kmh > 0.2:
            #done = False
            reward += 1 #-1
            # Reward lighter steering when moving
            if np.abs(action[1]) < 0.3:
                reward += 1
            elif np.abs(action[1]) > 0.5 and np.abs(action[1]) < 0.9:
                reward -= 0.1
            elif np.abs(action[1]) >= 0.9:
                reward -= 0.2
        elif kmh < 0.2:
            reward -= 0.1
        else:
            #print("Maybe never")
            reward += 0.01
            if np.abs(action[1]) < 0.3:
                reward += 0.12
            # Reduce score for heavy steering
            if np.abs(action[1]) > 0.5 and np.abs(action[1]) < 0.9:
                reward -= 0.17
            elif np.abs(action[1]) >= 0.9:
                reward -= 0.21
        '''
        '''
        rgb = BirdViewProducer.as_rgb(birdview)
        cv2.imshow('Frame',rgb)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
        '''
        if step < prep_limit:
            pass
        else:
            #print("HI")
            p.buffer.rewards.append(reward)
            p.buffer.is_terminals.append(done)
        if reward>=1:
            val_score+=1
        total_r += reward
        step += 1
        v = ego_vehicle.get_velocity()
        kmh = int(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2))
        #print(kmh,reward)
        #print(total_r,ci)
        '''
        if step % freq ==0 :
            print(step)
            p.update()

        if step % freq_n == 0:
            p.decay_action_std(0.05,0.1)


        '''
        world.tick()
    #total_r += step/10
    print(total_r,ci)
    '''
    if epoch > 50:

        if sum(total_reward_list[-50:])/len(total_reward_list[-50:]) > min_r_avg or sum(step_list) > decay_freq:
            decay_freq += decay_c
            #decay_c -= 50
            #print("Decaying:",p.action_std)
            p.decay_action_std(0.01,0.1)

            min_r_avg = sum(total_reward_list[-50:])/len(total_reward_list[-50:])
    '''
    '''
    if epoch%update_freq ==0 and epoch != 0:
            print(steer)
            print("Update with batches:",len(p.buffer.states))
            p.update()
            if steer_w > min_steer_w +0.1:
                steer_w -= steer_decay
    '''
    '''
    else:
            print(total_r,epoch,step)
            total_reward_list.append(total_r)
            epoch_list.append(epoch)
            step_list.append(step)
            continue
    #except Exception as e:
        #print("Error:",e)
        #pass
    '''
    '''
    #cv2.destroyAllWindows()
    #print(total_r,epoch,step)
    #total_reward_list.append(total_r)
    #epoch_list.append(epoch)
    #step_list.append(step)
    '''
    print("HI")

In [ ]:
val_score/50

In [ ]:
scenario = prepare_ngsim_scenario(client)
ids = list(world.get_actors())[0].id
carla.command.DestroyActor(ids)
world = client.get_world()
#spectator = world.get_spectator()
ego_vehicle = prepare_ego_vehicle(world)
scenario.reset(ego_vehicle)
done = False
c = world.tick()
way = scenario._veh.transform.as_carla_transform()
cmd_buffer = [0]
yaw_buffer = [0]
stab = False
aw = None
speed = 60
val = 0
steer = []
brake = []
throttle = []
step = 0
while not done:
    pid = VehiclePIDController(ego_vehicle,1.0)
    #speed = 90
    #print(way.location.x,way.location.y)
    k = pid.run_step(speed,way)
    #print(k.steer)
    if step == 0:
        k.steer = -0.1
    
    steer.append(k.steer)
    throttle.append(k.throttle)
    brake.append(k.brake)
    print(way.location.x,way.location.y)
    print(ego_vehicle.get_location().x,ego_vehicle.get_location().y)
    ego_vehicle.apply_control(carla.VehicleControl(throttle=k.throttle, steer=np.clip(k.steer,-0.9,0.9), brake = k.brake))
    r = scenario.step(ego_vehicle)
    print(r.reward,done,r.chauffeur_cmd)
    #way = scenario._target_lane_waypoint.transform
    val = r.chauffeur_cmd.value
    way,speed = way_cal(ego_vehicle,val,scenario)
    speed += 10
    print(_['on_target_lane'])
    #way = r.info["scenario_data"]["original_veh_transform"]
    cmd_buffer.append(r.chauffeur_cmd.value)
    yaw_buffer.append(ego_vehicle.get_transform().rotation.yaw)
    if len(cmd_buffer) > 10:
        if sum(cmd_buffer[-10:]) == 0 and _['on_target_lane'] and sum([abs(x) for x in yaw_buffer[-10:]])/10<=10:
            reward = 1 + random.random()/2
            done = True
    step +=1
    birdview = birdview_producer.produce(

            agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
            )
    rgb = BirdViewProducer.as_rgb(birdview)
    cv2.imshow('Frame',rgb)
    done = r.done
    if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
    c = world.tick()
#cv2.destroyAllWindows()

In [ ]:
_

In [ ]:
steer

In [ ]:
cmd_buffer

In [ ]:
scenario

In [ ]:
yaw_buffer

In [ ]:
cv2.destroyAllWindows()

In [ ]:
throttle

In [ ]:
for i in range(20):
    #scenario.reset(ego_vehicle)
    #client.apply_batch([carla.command.DestroyActor(x) for x in list(world.get_actors())])
    #world.tick()
    ids = list(world.get_actors())[0].id
    carla.command.DestroyActor(ids)
    world = client.get_world()
    #spectator = world.get_spectator()
    ego_vehicle = prepare_ego_vehicle(world)
    scenario.reset(ego_vehicle)
    done = False
    c = world.tick()
    way = ego_vehicle.get_transform()
    cmd_buffer = [0]
    yaw_buffer = [0]
    stab = False
    aw = None
    speed = 40
    val = 0
    while not done:
        pid = VehiclePIDController(ego_vehicle,0.9)
        speed = random.randint(30,50)
        k = pid.run_step(speed,way)
        ego_vehicle.apply_control(carla.VehicleControl(throttle=k.throttle, steer=k.steer, brake = k.brake))
        r = scenario.step(ego_vehicle)
        print(r.reward,done,r.chauffeur_cmd)
        #way = scenario._target_lane_waypoint.transform
        val = r.chauffeur_cmd.value
        way = way_cal(ego_vehicle,val)
        #way = r.info["scenario_data"]["original_veh_transform"]
        cmd_buffer.append(r.chauffeur_cmd.value)
        yaw_buffer.append(ego_vehicle.get_transform().rotation.yaw)
        if len(cmd_buffer) > 10:
            if sum(cmd_buffer[-10:]) == 0 and _['on_target_lane'] and sum([abs(x) for x in yaw_buffer[-10:]])/10<=10:
                reward = 1 + random.random()/2
                done = True

        birdview = birdview_producer.produce(

                agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
                )
        rgb = BirdViewProducer.as_rgb(birdview)
        cv2.imshow('Frame',rgb)
        done = r.done
        if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
        c = world.tick()
    cv2.destroyAllWindows()
    #time.sleep(3)

In [ ]:
m = Normal(torch.tensor([0.0]), torch.tensor([1.0]))


In [ ]:
m

In [ ]:
a = torch.arange(10).reshape(1,10)
b = torch.arange(100).reshape(1,100)

In [ ]:
a.shape

In [ ]:
torch.cat((a,b),1)

In [ ]:
print("Decaying:",p.action_std)

In [ ]:
_['scenario_data']['original_to_ego_distance']

In [ ]:
total_reward_list[-10:]

In [ ]:
val_reward = []
val_success = []
for i in range(50):
    t_clip_n = 0.0
    t_clip_p = 1.0

    s_clip_n = -1.0
    s_clip_p = 1.0

    step = 0
    scenario = prepare_ngsim_scenario(client,"Val")
    world = client.get_world()
    spectator = world.get_spectator()
    ego_vehicle = prepare_ego_vehicle(world)

    scenario.reset(ego_vehicle)
    c = world.tick()
    way = way_cal(ego_vehicle,val)
    done = False
    reward = 0
    val = 0
    cmd_buffer = [0]
    yaw_buffer = [0]
    while not done:

            birdview = birdview_producer.produce(
                agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
                )
            #a = birdview[0].reshape(1,186,150)
            #a = np.append(a,birdview[1].reshape(1,186,150),axis=0)
            #a = np.append(a,birdview[2].reshape(1,186,150),axis=0)
            #a = np.append(a,birdview[3].reshape(1,186,150),axis=0)
            #a = np.append(a,birdview[4].reshape(1,186,150),axis=0)
            #a = np.append(a,birdview[4].reshape(1,224,224),axis=0)
            #rgb = BirdViewProducer.as_rgb(birdview)/255.
            rgb = BirdViewProducer.as_rgb(birdview)
            #in_data = a.reshape(1,5,186,150)
            in_data = rgb.reshape(1,3,224,224)/255.

            with torch.no_grad():
                in_data = torch.FloatTensor(in_data).to(device)
                in_data = mobilenet.features(in_data)
                in_data = nn.AvgPool2d(7,7)(in_data)
                in_data = nn.Flatten()(in_data)
                in_data = in_data.detach().cpu().numpy()
            #in_data = mdl(in_data).numpy()
            #in_data = in_data.reshape(in_data.shape[0],62720)
            #in_data = in_data.reshape((1,5,186,150))
            #in_data = input_data.reshape((1,3,320,320))
            #print(in_data.shape)
            if val == 0 or val == 1:
                inps= [0,0,1]
            if val == 2 or val == 5:
                inps = [0,1,0]
            if val == 4 or val ==3:
                inps = [1,0,0]
            inps = np.array(inps)
            ap = np.concatenate((in_data[0],inps)).reshape(1,1283)

            action = p.select_action(ap)
            #print(action)
            #action = action*2 - 1
            #steer = action[0]
            #_speed = np.clip(action[0], -1,1)
            #speed = ((_speed + 1)/2)*50 + 10



            #pid = VehiclePIDController(ego_vehicle)
            #k = pid.run_step(scenario._veh.speed,way)
            #throttle = k.throttle
            #brake = k.brake
            d_action = action_space[action]
            
            steer = action[1]
            avg_steer = steer
            throttle = 0.0
            brake = 0.0
            if d_action[0]<0:
                brake = d_action[0]
            else:
                throttle = d_action[0]
            ego_vehicle.apply_control(carla.VehicleControl(throttle=throttle, steer=np.clip(avg_steer, s_clip_n, s_clip_p),brake=brake))

            #ego_vehicle.apply_control(carla.VehicleControl(throttle=np.clip(throttle, t_clip_n, t_clip_p), steer=np.clip(action[1], s_clip_n, s_clip_p)))#,brake=np.clip(brake, 0.0, 1.0)))



            cmd, reward, done, _ = scenario.step(ego_vehicle)
            '''
            if reward < 0 :
                reward = -0.1
            '''



            #print(reward, cmd, _['scenario_data']['original_to_ego_distance'], throttle,steer)

            val = cmd.value
            cmd_buffer.append(val)
            yaw_buffer.append(ego_vehicle.get_transform().rotation.yaw)
            '''
            if len(cmd_buffer) > 5:
                if sum(cmd_buffer[-5:]) == 0 and _['on_target_lane'] and abs(yaw_buffer[-1])<=10:
                    reward = 1.1
                    done = True
            '''
            
            if reward >=0.9:
                print("Close to win")
            if reward >= 1:
                print("Its a win")
                #p.save("WinModel"+str(epoch)+".mdl")
            #way = _["scenario_data"]["original_veh_transform"]
            way = way_cal(ego_vehicle,val)
            #way = scenario._target_lane_waypoint.transform 
            '''
            rgb = BirdViewProducer.as_rgb(birdview)
            cv2.imshow('Frame',rgb)
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
            '''
            c = world.tick()
    if reward >0.9:
        val_reward.append(reward)
        val_success.append(1)
    else:
        val_reward.append(reward)
        val_success.append(0)

In [ ]:
_['on_target_lane']

In [ ]:
sum(val_success)/len(val_success)

In [ ]:
ego_vehicle.get_transform().rotation.yaw

In [ ]:
_['scenario_data']['original_veh_transform'].rotation.yaw

In [ ]:
p.load("Model_CHK_TorchEasy.mdl")

In [ ]:
p.action_std = 0.02

In [ ]:
p.decay_action_std(0.01,0.01)

In [ ]:
p.action_std

In [ ]:
reward

In [ ]:
cv2.destroyAllWindows()

In [ ]:
int(2.5e5)

In [ ]:
scenario.reset(ego_vehicle)

In [ ]:
scenario.step(ego_vehicle)

In [ ]:
ego_vehicle.get_location().x

In [ ]:
ego_vehicle = prepare_ego_vehicle(world)

In [ ]:
world = client.get_world()

In [ ]:
world.tick()

In [ ]:
ego_vehicle.apply_control(carla.VehicleControl(throttle=0.7, steer=0.1))

In [ ]:
torch.seed()

In [ ]:
History = [epoch_list,total_reward_list,step_list]

In [ ]:
len(epoch_list)

In [ ]:
len(epoch_list)

In [ ]:
import pickle
f = open("History00.pkl",'wb')
pickle.dump(History,f)
f.close()

In [ ]:
p.save("Model_Best.mdl")

In [ ]:
len(total_reward_list)

In [ ]:
epoch_list = epoch_list[500:]
total_reward_list = total_reward_list[500:]
step_list = step_list[500:]

In [ ]:
#plt.ylim(-2,1)
plt.plot(epoch_list,total_reward_list)

In [ ]:
count = 0
for i in total_reward_list[400:600]:
    if i>0:
        count +=1
        

In [ ]:
count/200

In [ ]:
cv2.destroyAllWindows()

In [ ]:
320*320*3

In [ ]:
input_data

In [ ]:
in_data

In [ ]:
in_data = input_data.reshape((1,3,320,320))

In [ ]:
del p

In [ ]:
p.select_action(in_data)

In [ ]:
p.buffer.actions

In [ ]:
p.buffer.rewards

In [ ]:
import glob
import pickle
import numpy as np
import matplotlib.pyplot as plt
idx = '2'
l = glob.glob('Hm/*.pkl')
epoch_list_0 = [] 

hist = {}
for i in l:
    f = open(i,'rb')
    s = pickle.load(f)
    f.close()
    n = i.split('/')[1]
    n = n.split('.')[0]
    n = n.split('_')
    if n[2] in hist.keys():
        pass
    else:
        hist[n[2]] = {}
    hist[n[2]][n[1]] = s 
    
for i in sorted([int(x) for x in hist[idx].keys()]):
    epoch_list_0 += hist[idx][str(i)][0]
reward_list_0 = []

for i in sorted([int(x) for x in hist[idx].keys()]):
    reward_list_0 += hist[idx][str(i)][1]
step_list_0 = []
for i in sorted([int(x) for x in hist[idx].keys()]):
    step_list_0 += hist[idx][str(i)][2]
final_epoch_list = epoch_list_0
final_reward_list = reward_list_0
final_step_list = step_list_0
val_list = []
ep_list = []
for i in sorted([int(x) for x in hist[idx].keys()]):
    ep_list.append(i)
    val_list.append(hist[idx][str(i)][3])
epochs = list(range(len(val_list)))
ids =sorted([int(x) for x in hist[idx].keys()])
train_score = []
average_train = []
for i in epochs:
    c = ids[i]
    arr = hist[idx][str(c)][1]
    count = 0
    sme = 0
    for j in arr:
        sme += j
        if j >0:
            count+=1
    average_train.append(sme/len(arr))
    train_score.append(count/len(arr))

In [ ]:
plt.xlabel("Epochs")
plt.ylabel("Reward Sum")
plt.plot(final_epoch_list,final_reward_list)

In [ ]:
plt.xlabel("Epochs")
plt.ylabel("Step size")
plt.plot(final_epoch_list,final_step_list)

In [ ]:
#plt.ylim(0,1.0)
#plt.xlim(0,5000)
plt.xlabel("Epochs")
plt.ylabel("success rate")
plt.plot(epochs,train_score)
plt.plot(epochs,val_list)

In [ ]:
#plt.xlim(0,3000)
#plt.ylim(-1,1)
plt.xlabel("Epochs")
plt.ylabel("TrainRewardAvg")
plt.plot(epochs,average_train)
#plt.plot(epochs,val_list)

In [ ]:
sorted([int(x) for x in hist[idx].keys()])[172]

In [ ]:
epochs[172]

In [ ]:
val_list[172]

In [ ]:
val_list[150:200][20:25]

In [ ]:
train_score[150:200][20:25]

In [ ]:
buff = [True,True,False,True]

In [ ]:
all(item == True for item in buff)

In [ ]:
import glob


In [ ]:
l = glob.glob('He/*.pkl')

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
l

In [ ]:
i.split('/')

In [ ]:
hist = {}
for i in l:
    f = open(i,'rb')
    s = pickle.load(f)
    f.close()
    n = i.split('/')[1]
    n = n.split('.')[0]
    n = n.split('_')
    if n[2] in hist.keys():
        pass
    else:
        hist[n[2]] = {}
    hist[n[2]][n[1]] = s 

In [ ]:
hist

In [ ]:
if ('w','2000') in hist.keys():
    print("HI")

In [ ]:
hist

In [ ]:
epoch_list_0 = [] 

In [ ]:
sorted([int(x) for x in hist['0'].keys()])

In [ ]:
for i in sorted([int(x) for x in hist['2'].keys()]):
    epoch_list_0 += hist['2'][str(i)][0]

In [ ]:
hist['0']['500'][0]

In [ ]:
len(epoch_list_0)

In [ ]:
reward_list_0 = []

for i in sorted([int(x) for x in hist['2'].keys()]):
    reward_list_0 += hist['2'][str(i)][1]

In [ ]:
reward_list_0

In [ ]:
step_list_0 = []
for i in sorted([int(x) for x in hist['2'].keys()]):
    step_list_0 += hist['2'][str(i)][2]

In [ ]:
step_list_0

In [ ]:
epoch_list_1 = []
for i in sorted([int(x) for x in hist['1'].keys()]):
    epoch_list_1 += hist['1'][str(i)][0]
reward_list_1 = []
for i in sorted([int(x) for x in hist['1'].keys()]):
    reward_list_1 += hist['1'][str(i)][1]
step_list_1 = []
for i in sorted([int(x) for x in hist['1'].keys()]):
    step_list_1 += hist['1'][str(i)][2]

In [ ]:
final_epoch_list = epoch_list_0 + [x + len(epoch_list_0) for x in epoch_list_1]
final_reward_list = reward_list_0 + reward_list_1
final_step_list = step_list_0 + step_list_1

In [ ]:
final_epoch_list = epoch_list_0
final_reward_list = reward_list_0
final_step_list = step_list_0

In [ ]:
print(len(final_epoch_list))
print(len(final_reward_list))
len(final_step_list)

In [ ]:
plt.xlabel("Epochs")
plt.ylabel("Reward Sum")
plt.plot(final_epoch_list,final_reward_list)

In [ ]:
plt.xlabel("Epochs")
plt.ylabel("Step size")
plt.plot(final_epoch_list,final_step_list)

In [ ]:
sorted([int(x) for x in hist['1'].keys()])[28]

In [ ]:
sorted(hist['0'].keys())

In [ ]:
val_list = []
ep_list = []
for i in sorted([int(x) for x in hist['2'].keys()]):
    ep_list.append(i)
    val_list.append(hist['2'][str(i)][3])

In [ ]:
epochs = list(range(len(val_list)))

In [ ]:
plt.ylim(0,1.0)
#plt.xlim(0,5000)
plt.xlabel("Epochs")
plt.ylabel("success rate")
plt.plot(epochs,train_score)
plt.plot(epochs,val_list)

In [ ]:
plt.ylim(0,1.0)
#plt.xlim(0,5000)
plt.xlabel("Epochs")
plt.ylabel("success rate")
plt.plot(epochs,train_score)
plt.plot(epochs,val_list)

In [ ]:
ids =sorted([int(x) for x in hist['2'].keys()])

In [ ]:
c = ids[0]

In [ ]:
hist['0'][str(c)][1]

In [ ]:
train_score = []

for i in epochs:
    c = ids[i]
    arr = hist['2'][str(c)][1]
    count = 0
    for j in arr:
        if j >0:
            count+=1
    train_score.append(count/len(arr))

In [ ]:
train_score

In [ ]:
final_epoch_list

In [ ]:
epochs[310:320]

In [ ]:
val_list[310:330]

In [ ]:
len(val_list)

In [ ]:
np.mean(final_reward_list)

In [ ]:
np.mean(final_step_list)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(np.array(epoch_list).reshape(-1,1),np.array(total_reward_list))

In [ ]:
model.coef_

In [ ]:
x = list(range(0,7000))

In [ ]:
y = model.predict(np.array(x).reshape(-1,1))

In [ ]:
x = epoch_list
y= total_reward_list

In [ ]:
plt.xlabel("Epochs")
plt.ylabel("Rewards")
plt.ylim(-2.0,2.0)
plt.plot(x,y)

In [ ]:
plt.xlabel("Epochs")
plt.ylabel("Steps")
plt.ylim(0.0,10.0)
plt.plot(x,y)

In [ ]:
f = open("History_5500_1.pkl",'rb')
s = pickle.load(f)
f.close()

In [ ]:
e = s[0]
r = s[1]
t = s[2]

In [ ]:
plt.plot(e,r)

In [ ]:
plt.plot(e,t)